In [2]:
import yfinance as yf
import pandas as pd

def init_pipeline(symbol, api_key=None):
    """api_key parameter kept for compatibility but not used"""
    ticker = yf.Ticker(symbol)
    return ticker, symbol

# No API key needed for Yahoo Finance
fd, symbol = init_pipeline("GOOG")

# Get financial statements (annual by default, use period="quarterly" for quarterly)
is_df = fd.financials.T  # Income statement
bs_df = fd.balance_sheet.T  # Balance sheet
cf_df = fd.cashflow.T  # Cash flow

def clean_and_trim_5y(df):
    """
    Clean Yahoo Finance financial statements
    and keep the most recent 5 years.
    """
    df = df.copy()
    df = df.sort_index().tail(5)  # Yahoo Finance uses datetime index
    df.reset_index(inplace=True)
    df.rename(columns={df.columns[0]: "fiscalDateEnding"}, inplace=True)
    return df

# Apply cleaning
is_df = clean_and_trim_5y(is_df)
bs_df = clean_and_trim_5y(bs_df)
cf_df = clean_and_trim_5y(cf_df)

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np

def init_pipeline(symbol, api_key=None):
    """api_key parameter kept for compatibility but not used"""
    ticker = yf.Ticker(symbol)
    return ticker, symbol

def clean_and_trim_5y(df):
    """
    Clean Yahoo Finance financial statements
    and keep the most recent 5 years.
    """
    df = df.copy()
    df = df.sort_index().tail(5)  # Yahoo Finance uses datetime index
    df.reset_index(inplace=True)
    df.rename(columns={df.columns[0]: "fiscalDateEnding"}, inplace=True)
    return df

def safe_divide(numerator, denominator, default=np.nan):
    """Safely divide two numbers, returning default if denominator is 0 or None"""
    if denominator is None or denominator == 0 or pd.isna(denominator):
        return default
    if numerator is None or pd.isna(numerator):
        return default
    return numerator / denominator

def get_value(df, col_name, idx=0):
    """Safely get a value from dataframe"""
    if col_name not in df.columns:
        return None
    val = df.iloc[idx][col_name] if idx < len(df) else None
    return val if not pd.isna(val) else None

def get_avg(current, previous):
    """Calculate average, using current if previous is not available"""
    if previous is None or pd.isna(previous):
        return current
    if current is None or pd.isna(current):
        return previous
    return (current + previous) / 2

def calculate_all_ratios(symbol):
    """
    Calculate all financial ratios for a given stock symbol
    Returns a dictionary of DataFrames with ratios by category
    """
    # Initialize
    fd, symbol = init_pipeline(symbol)
    
    # Get financial statements (quarterly)
    print(f"Fetching data for {symbol}...")
    is_df = fd.quarterly_financials.T  # Income statement
    bs_df = fd.quarterly_balance_sheet.T  # Balance sheet
    cf_df = fd.quarterly_cashflow.T  # Cash flow
    
    # Clean data
    is_df = clean_and_trim_5y(is_df)
    bs_df = clean_and_trim_5y(bs_df)
    cf_df = clean_and_trim_5y(cf_df)
    
    # Get market data
    info = fd.info
    current_price = info.get('currentPrice', info.get('regularMarketPrice'))
    shares_outstanding = info.get('sharesOutstanding')
    market_cap = info.get('marketCap')
    
    # Prepare results storage
    results = []
    
    # Process each quarter
    for i in range(len(is_df)):
        quarter_ratios = {}
        quarter_ratios['fiscalDateEnding'] = is_df.iloc[i]['fiscalDateEnding']
        
        # ============ EXTRACT DATA FROM STATEMENTS ============
        
        # Income Statement
        revenue = get_value(is_df, 'Total Revenue', i)
        gross_profit = get_value(is_df, 'Gross Profit', i)
        cost_of_revenue = get_value(is_df, 'Cost Of Revenue', i)
        operating_income = get_value(is_df, 'Operating Income', i)
        net_income = get_value(is_df, 'Net Income', i)
        ebit = get_value(is_df, 'EBIT', i)
        ebitda = get_value(is_df, 'EBITDA', i)
        pretax_income = get_value(is_df, 'Pretax Income', i)
        tax_expense = get_value(is_df, 'Tax Provision', i)
        interest_expense = get_value(is_df, 'Interest Expense', i)
        
        # Balance Sheet
        total_assets = get_value(bs_df, 'Total Assets', i)
        current_assets = get_value(bs_df, 'Current Assets', i)
        cash = get_value(bs_df, 'Cash And Cash Equivalents', i)
        if cash is None:
            cash = get_value(bs_df, 'Cash Cash Equivalents And Short Term Investments', i)
        inventory = get_value(bs_df, 'Inventory', i)
        receivables = get_value(bs_df, 'Receivables', i)
        if receivables is None:
            receivables = get_value(bs_df, 'Accounts Receivable', i)
        
        total_liabilities = get_value(bs_df, 'Total Liabilities Net Minority Interest', i)
        current_liabilities = get_value(bs_df, 'Current Liabilities', i)
        total_debt = get_value(bs_df, 'Total Debt', i)
        if total_debt is None:
            long_term = get_value(bs_df, 'Long Term Debt', i) or 0
            short_term = get_value(bs_df, 'Current Debt', i) or 0
            total_debt = long_term + short_term if (long_term or short_term) else None
        
        total_equity = get_value(bs_df, 'Stockholders Equity', i)
        if total_equity is None:
            total_equity = get_value(bs_df, 'Total Equity Gross Minority Interest', i)
        
        # Cash Flow Statement
        operating_cf = get_value(cf_df, 'Operating Cash Flow', i)
        capex = get_value(cf_df, 'Capital Expenditure', i)
        
        # Previous period values for averaging
        prev_assets = get_value(bs_df, 'Total Assets', i+1) if i+1 < len(bs_df) else None
        prev_equity = get_value(bs_df, 'Stockholders Equity', i+1) if i+1 < len(bs_df) else None
        if prev_equity is None:
            prev_equity = get_value(bs_df, 'Total Equity Gross Minority Interest', i+1) if i+1 < len(bs_df) else None
        prev_inventory = get_value(bs_df, 'Inventory', i+1) if i+1 < len(bs_df) else None
        prev_receivables = get_value(bs_df, 'Receivables', i+1) if i+1 < len(bs_df) else None
        if prev_receivables is None:
            prev_receivables = get_value(bs_df, 'Accounts Receivable', i+1) if i+1 < len(bs_df) else None
        prev_debt = get_value(bs_df, 'Total Debt', i+1) if i+1 < len(bs_df) else None
        prev_cash = get_value(bs_df, 'Cash And Cash Equivalents', i+1) if i+1 < len(bs_df) else None
        if prev_cash is None:
            prev_cash = get_value(bs_df, 'Cash Cash Equivalents And Short Term Investments', i+1) if i+1 < len(bs_df) else None
        
        # Calculate averages
        avg_assets = get_avg(total_assets, prev_assets)
        avg_equity = get_avg(total_equity, prev_equity)
        avg_inventory = get_avg(inventory, prev_inventory)
        avg_receivables = get_avg(receivables, prev_receivables)
        
        # ============ PROFITABILITY RATIOS ============
        
        # Gross Margin
        if gross_profit is not None:
            quarter_ratios['Gross Margin (%)'] = safe_divide(gross_profit, revenue, 0) * 100
        elif cost_of_revenue is not None and revenue is not None:
            quarter_ratios['Gross Margin (%)'] = safe_divide(revenue - cost_of_revenue, revenue, 0) * 100
        else:
            quarter_ratios['Gross Margin (%)'] = np.nan
        
        quarter_ratios['Operating Margin (%)'] = safe_divide(operating_income, revenue, 0) * 100
        quarter_ratios['Net Margin (%)'] = safe_divide(net_income, revenue, 0) * 100
        quarter_ratios['EBITDA Margin (%)'] = safe_divide(ebitda, revenue, 0) * 100
        
        # ROA with average assets
        quarter_ratios['ROA (%)'] = safe_divide(net_income, avg_assets, 0) * 100
        
        # ROE with average equity
        quarter_ratios['ROE (%)'] = safe_divide(net_income, avg_equity, 0) * 100
        
        # ROIC
        # Calculate tax rate
        tax_rate = safe_divide(tax_expense, pretax_income, 0.25)  # Default to 25% if unavailable
        if tax_rate < 0 or tax_rate > 1:
            tax_rate = 0.25
        
        # Calculate invested capital
        current_net_debt = (total_debt or 0) - (cash or 0)
        current_ic = current_net_debt + (total_equity or 0)
        
        if prev_debt is not None and prev_cash is not None and prev_equity is not None:
            prev_net_debt = prev_debt - prev_cash
            prev_ic = prev_net_debt + prev_equity
            invested_capital = (current_ic + prev_ic) / 2
        else:
            invested_capital = current_ic
        
        nopat = (ebit or 0) * (1 - tax_rate)
        quarter_ratios['ROIC (%)'] = safe_divide(nopat, invested_capital, 0) * 100
        
        # ============ LIQUIDITY RATIOS ============
        
        quarter_ratios['Current Ratio'] = safe_divide(current_assets, current_liabilities)
        quarter_ratios['Quick Ratio'] = safe_divide((current_assets or 0) - (inventory or 0), current_liabilities)
        quarter_ratios['Cash Ratio'] = safe_divide(cash, current_liabilities)
        quarter_ratios['Operating Cash Flow Ratio'] = safe_divide(operating_cf, current_liabilities)
        
        # ============ LEVERAGE RATIOS ============
        
        quarter_ratios['Debt-to-Equity Ratio'] = safe_divide(total_debt, total_equity)
        quarter_ratios['Debt-to-Assets Ratio'] = safe_divide(total_debt, total_assets)
        quarter_ratios['Equity Multiplier'] = safe_divide(total_assets, total_equity)
        
        if interest_expense is not None:
            quarter_ratios['Interest Coverage Ratio'] = safe_divide(ebit, abs(interest_expense))
        else:
            quarter_ratios['Interest Coverage Ratio'] = np.nan
        
        quarter_ratios['Operating Cash Flow to Debt Ratio'] = safe_divide(operating_cf, total_debt)
        
        if interest_expense is not None:
            quarter_ratios['EBITDA Interest Coverage Ratio'] = safe_divide(ebitda, abs(interest_expense))
        else:
            quarter_ratios['EBITDA Interest Coverage Ratio'] = np.nan
        
        # ============ EFFICIENCY RATIOS ============
        
        quarter_ratios['Asset Turnover'] = safe_divide(revenue, avg_assets)
        quarter_ratios['Inventory Turnover'] = safe_divide(cost_of_revenue, avg_inventory)
        
        inventory_turnover = safe_divide(cost_of_revenue, avg_inventory)
        quarter_ratios['Days Inventory Outstanding (DIO)'] = safe_divide(365, inventory_turnover)
        
        quarter_ratios['Receivables Turnover'] = safe_divide(revenue, avg_receivables)
        
        receivables_turnover = safe_divide(revenue, avg_receivables)
        quarter_ratios['Days Sales Outstanding (DSO)'] = safe_divide(365, receivables_turnover)
        
        # ============ CASH FLOW RATIOS ============
        
        fcf = (operating_cf or 0) - abs(capex or 0)
        quarter_ratios['Free Cash Flow'] = fcf
        
        quarter_ratios['Operating Cash Flow Margin (%)'] = safe_divide(operating_cf, revenue, 0) * 100
        quarter_ratios['Free Cash Flow Margin (%)'] = safe_divide(fcf, revenue, 0) * 100
        quarter_ratios['Cash Flow to Net Income'] = safe_divide(operating_cf, net_income)
        quarter_ratios['Capex to Operating Cash Flow Ratio (%)'] = safe_divide(abs(capex or 0), operating_cf, 0) * 100
        
        # ============ VALUATION RATIOS ============
        
        if shares_outstanding is not None:
            eps = safe_divide(net_income, shares_outstanding)
            quarter_ratios['EPS'] = eps
            quarter_ratios['Price-to-Earnings (P/E)'] = safe_divide(current_price, eps) if current_price else np.nan
            quarter_ratios['Book Value Per Share'] = safe_divide(total_equity, shares_outstanding)
        else:
            quarter_ratios['EPS'] = np.nan
            quarter_ratios['Price-to-Earnings (P/E)'] = np.nan
            quarter_ratios['Book Value Per Share'] = np.nan
        
        if market_cap is not None:
            quarter_ratios['Price-to-Book (P/B)'] = safe_divide(market_cap, total_equity)
            quarter_ratios['Price-to-Sales (P/S)'] = safe_divide(market_cap, revenue)
            
            enterprise_value = market_cap + (total_debt or 0) - (cash or 0)
            quarter_ratios['Enterprise Value (EV)'] = enterprise_value
            quarter_ratios['EV/EBITDA'] = safe_divide(enterprise_value, ebitda)
            quarter_ratios['EV/Sales'] = safe_divide(enterprise_value, revenue)
            quarter_ratios['Market Cap to FCF'] = safe_divide(market_cap, fcf)
        else:
            quarter_ratios['Price-to-Book (P/B)'] = np.nan
            quarter_ratios['Price-to-Sales (P/S)'] = np.nan
            quarter_ratios['Enterprise Value (EV)'] = np.nan
            quarter_ratios['EV/EBITDA'] = np.nan
            quarter_ratios['EV/Sales'] = np.nan
            quarter_ratios['Market Cap to FCF'] = np.nan
        
        results.append(quarter_ratios)
    
    # Convert to DataFrame
    ratios_df = pd.DataFrame(results)
    
    # Organize by category
    profitability_cols = ['fiscalDateEnding', 'Gross Margin (%)', 'Operating Margin (%)', 
                          'Net Margin (%)', 'EBITDA Margin (%)', 'ROA (%)', 'ROE (%)', 'ROIC (%)']
    liquidity_cols = ['fiscalDateEnding', 'Current Ratio', 'Quick Ratio', 'Cash Ratio', 
                      'Operating Cash Flow Ratio']
    leverage_cols = ['fiscalDateEnding', 'Debt-to-Equity Ratio', 'Debt-to-Assets Ratio', 
                     'Equity Multiplier', 'Interest Coverage Ratio', 
                     'Operating Cash Flow to Debt Ratio', 'EBITDA Interest Coverage Ratio']
    efficiency_cols = ['fiscalDateEnding', 'Asset Turnover', 'Inventory Turnover', 
                       'Days Inventory Outstanding (DIO)', 'Receivables Turnover', 
                       'Days Sales Outstanding (DSO)']
    cashflow_cols = ['fiscalDateEnding', 'Free Cash Flow', 'Operating Cash Flow Margin (%)', 
                     'Free Cash Flow Margin (%)', 'Cash Flow to Net Income', 
                     'Capex to Operating Cash Flow Ratio (%)']
    valuation_cols = ['fiscalDateEnding', 'EPS', 'Price-to-Earnings (P/E)', 'Price-to-Book (P/B)', 
                      'Book Value Per Share', 'Price-to-Sales (P/S)', 'Enterprise Value (EV)', 
                      'EV/EBITDA', 'EV/Sales', 'Market Cap to FCF']
    
    return {
        'all_ratios': ratios_df,
        'profitability': ratios_df[[col for col in profitability_cols if col in ratios_df.columns]],
        'liquidity': ratios_df[[col for col in liquidity_cols if col in ratios_df.columns]],
        'leverage': ratios_df[[col for col in leverage_cols if col in ratios_df.columns]],
        'efficiency': ratios_df[[col for col in efficiency_cols if col in ratios_df.columns]],
        'cashflow': ratios_df[[col for col in cashflow_cols if col in ratios_df.columns]],
        'valuation': ratios_df[[col for col in valuation_cols if col in ratios_df.columns]]
    }

# ============ USAGE EXAMPLE ============
if __name__ == "__main__":
    symbol = "GOOG"
    
    # Calculate all ratios
    ratios = calculate_all_ratios(symbol)
    
    # Display results
    print("\n" + "="*80)
    print(f"FINANCIAL RATIOS FOR {symbol}")
    print("="*80)
    
    print("\n### PROFITABILITY RATIOS ###")
    print(ratios['profitability'].to_string(index=False))
    
    print("\n### LIQUIDITY RATIOS ###")
    print(ratios['liquidity'].to_string(index=False))
    
    print("\n### LEVERAGE RATIOS ###")
    print(ratios['leverage'].to_string(index=False))
    
    print("\n### EFFICIENCY RATIOS ###")
    print(ratios['efficiency'].to_string(index=False))
    
    print("\n### CASH FLOW RATIOS ###")
    print(ratios['cashflow'].to_string(index=False))
    
    print("\n### VALUATION RATIOS ###")
    print(ratios['valuation'].to_string(index=False))
    
    # Export to Excel
    with pd.ExcelWriter(f'{symbol}_financial_ratios.xlsx') as writer:
        ratios['all_ratios'].to_excel(writer, sheet_name='All Ratios', index=False)
        ratios['profitability'].to_excel(writer, sheet_name='Profitability', index=False)
        ratios['liquidity'].to_excel(writer, sheet_name='Liquidity', index=False)
        ratios['leverage'].to_excel(writer, sheet_name='Leverage', index=False)
        ratios['efficiency'].to_excel(writer, sheet_name='Efficiency', index=False)
        ratios['cashflow'].to_excel(writer, sheet_name='Cash Flow', index=False)
        ratios['valuation'].to_excel(writer, sheet_name='Valuation', index=False)
    
    print(f"\n✓ Ratios exported to {symbol}_financial_ratios.xlsx")

Fetching data for GOOG...

FINANCIAL RATIOS FOR GOOG

### PROFITABILITY RATIOS ###
fiscalDateEnding  Gross Margin (%)  Operating Margin (%)  Net Margin (%)  EBITDA Margin (%)  ROA (%)  ROE (%)  ROIC (%)
      2024-09-30             58.68                 32.31           29.80              40.50     5.97     8.23      8.13
      2024-12-31             57.90                 32.11           27.51              37.84     5.73     7.92      7.90
      2025-03-31             59.70                 33.92           38.28              51.32     7.07     9.75      9.56
      2025-06-30             59.51                 32.43           29.24              40.64     5.43     7.52      7.33
      2025-09-30             59.58                 30.51           34.18              48.60     6.52     9.04      8.83

### LIQUIDITY RATIOS ###
fiscalDateEnding  Current Ratio  Quick Ratio  Cash Ratio  Operating Cash Flow Ratio
      2024-09-30           1.95         1.95        0.25                       0.38
   

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np

def calculate_dcf_valuation(symbol, forecast_years=5):
    """
    Calculate DCF valuation with minimal assumptions.
    Most parameters are calculated from historical data.
    
    Returns: dictionary with valuation results and all assumptions used
    """
    
    print(f"\n{'='*80}")
    print(f"DCF VALUATION FOR {symbol}")
    print(f"{'='*80}\n")
    
    # Fetch data
    ticker = yf.Ticker(symbol)
    
    # Get financial statements
    cf_df = ticker.quarterly_cashflow.T
    bs_df = ticker.quarterly_balance_sheet.T
    is_df = ticker.quarterly_financials.T
    
    # Sort by date
    cf_df = cf_df.sort_index()
    bs_df = bs_df.sort_index()
    is_df = is_df.sort_index()
    
    # Get market data
    info = ticker.info
    current_price = info.get('currentPrice', info.get('regularMarketPrice'))
    shares_outstanding = info.get('sharesOutstanding')
    market_cap = info.get('marketCap')
    beta = info.get('beta', 1.0)  # Default to 1.0 if not available
    
    print(f"Current Price: ${current_price:.2f}")
    print(f"Shares Outstanding: {shares_outstanding:,.0f}")
    print(f"Market Cap: ${market_cap:,.0f}\n")
    
    # ============================================================
    # STEP 1: CALCULATE HISTORICAL FREE CASH FLOWS
    # ============================================================
    
    print("STEP 1: Historical Free Cash Flow Analysis")
    print("-" * 80)
    
    # Get last 8 quarters for TTM calculation
    operating_cf_list = []
    capex_list = []
    fcf_list = []
    
    for i in range(min(8, len(cf_df))):
        ocf = cf_df.iloc[-(i+1)].get('Operating Cash Flow')
        capex = cf_df.iloc[-(i+1)].get('Capital Expenditure', 0)
        
        if pd.notna(ocf):
            operating_cf_list.append(ocf)
            capex_list.append(abs(capex) if pd.notna(capex) else 0)
            fcf_list.append(ocf - abs(capex) if pd.notna(capex) else ocf)
    
    # Calculate TTM (Trailing Twelve Months) FCF
    ttm_operating_cf = sum(operating_cf_list[:4])  # Last 4 quarters
    ttm_capex = sum(capex_list[:4])
    ttm_fcf = ttm_operating_cf - ttm_capex
    
    print(f"TTM Operating Cash Flow: ${ttm_operating_cf:,.0f}")
    print(f"TTM CapEx: ${ttm_capex:,.0f}")
    print(f"TTM Free Cash Flow: ${ttm_fcf:,.0f}")
    
    # Calculate historical FCF growth rate
    if len(fcf_list) >= 8:
        # Compare last 4 quarters vs previous 4 quarters
        recent_fcf = sum(fcf_list[:4])
        previous_fcf = sum(fcf_list[4:8])
        
        if previous_fcf > 0:
            fcf_growth_rate = (recent_fcf / previous_fcf) - 1
        else:
            fcf_growth_rate = 0.10  # Default to 10% if previous was negative
    else:
        fcf_growth_rate = 0.10  # Default to 10%
    
    # Cap growth rate at reasonable levels
    fcf_growth_rate = max(min(fcf_growth_rate, 0.30), -0.10)  # Between -10% and 30%
    
    print(f"Historical FCF Growth Rate: {fcf_growth_rate*100:.2f}%")
    
    # Calculate average CapEx as % of Operating CF for forecasting
    capex_to_ocf_ratio = ttm_capex / ttm_operating_cf if ttm_operating_cf > 0 else 0.20
    print(f"CapEx / Operating CF Ratio: {capex_to_ocf_ratio*100:.2f}%\n")
    
    # ============================================================
    # STEP 2: CALCULATE WACC (Weighted Average Cost of Capital)
    # ============================================================
    
    print("STEP 2: Calculating WACC")
    print("-" * 80)
    
    # Get balance sheet items
    total_debt = bs_df.iloc[-1].get('Total Debt')
    if pd.isna(total_debt):
        long_term_debt = bs_df.iloc[-1].get('Long Term Debt', 0)
        short_term_debt = bs_df.iloc[-1].get('Current Debt', 0)
        total_debt = (long_term_debt if pd.notna(long_term_debt) else 0) + \
                     (short_term_debt if pd.notna(short_term_debt) else 0)
    
    cash = bs_df.iloc[-1].get('Cash And Cash Equivalents')
    if pd.isna(cash):
        cash = bs_df.iloc[-1].get('Cash Cash Equivalents And Short Term Investments', 0)
    
    print(f"Total Debt: ${total_debt:,.0f}")
    print(f"Cash: ${cash:,.0f}")
    
    # Calculate Cost of Debt from historical data
    interest_expense_list = []
    debt_list = []
    
    for i in range(min(4, len(is_df))):
        interest = is_df.iloc[-(i+1)].get('Interest Expense')
        debt = bs_df.iloc[-(i+1)].get('Total Debt')
        
        if pd.notna(interest) and pd.notna(debt) and debt > 0:
            interest_expense_list.append(abs(interest))
            debt_list.append(debt)
    
    if interest_expense_list and debt_list:
        avg_interest = sum(interest_expense_list)
        avg_debt = np.mean(debt_list)
        cost_of_debt = avg_interest / avg_debt if avg_debt > 0 else 0.04
    else:
        cost_of_debt = 0.04  # Default to 4%
    
    print(f"Cost of Debt: {cost_of_debt*100:.2f}%")
    
    # Calculate Tax Rate from historical data
    tax_expense_list = []
    pretax_income_list = []
    
    for i in range(min(4, len(is_df))):
        tax = is_df.iloc[-(i+1)].get('Tax Provision')
        pretax = is_df.iloc[-(i+1)].get('Pretax Income')
        
        if pd.notna(tax) and pd.notna(pretax) and pretax > 0:
            tax_expense_list.append(tax)
            pretax_income_list.append(pretax)
    
    if tax_expense_list and pretax_income_list:
        total_tax = sum(tax_expense_list)
        total_pretax = sum(pretax_income_list)
        tax_rate = total_tax / total_pretax if total_pretax > 0 else 0.21
        tax_rate = max(min(tax_rate, 0.35), 0.10)  # Between 10% and 35%
    else:
        tax_rate = 0.21  # Default to 21% (US corporate rate)
    
    print(f"Effective Tax Rate: {tax_rate*100:.2f}%")
    
    # Risk-free rate - use 10-year Treasury yield
    # Fetch current 10-year Treasury rate
    try:
        tnx = yf.Ticker("^TNX")
        risk_free_rate = tnx.info.get('regularMarketPrice', 4.5) / 100  # TNX is in percentage
    except:
        risk_free_rate = 0.045  # Default to 4.5%
    
    print(f"Risk-Free Rate (10Y Treasury): {risk_free_rate*100:.2f}%")
    
    # Market Risk Premium - use historical average
    market_risk_premium = 0.06  # 6% is historical average for US market
    print(f"Market Risk Premium (Historical Avg): {market_risk_premium*100:.2f}%")
    
    # Beta from yfinance
    print(f"Beta: {beta:.2f}")
    
    # Calculate Cost of Equity using CAPM
    cost_of_equity = risk_free_rate + (beta * market_risk_premium)
    print(f"Cost of Equity (CAPM): {cost_of_equity*100:.2f}%")
    
    # Calculate WACC
    market_value_equity = market_cap
    market_value_debt = total_debt  # Using book value as proxy
    total_value = market_value_equity + market_value_debt
    
    weight_equity = market_value_equity / total_value if total_value > 0 else 1
    weight_debt = market_value_debt / total_value if total_value > 0 else 0
    
    wacc = (weight_equity * cost_of_equity) + (weight_debt * cost_of_debt * (1 - tax_rate))
    
    print(f"\nWeight of Equity: {weight_equity*100:.2f}%")
    print(f"Weight of Debt: {weight_debt*100:.2f}%")
    print(f"WACC: {wacc*100:.2f}%\n")
    
    # ============================================================
    # STEP 3: FORECAST FREE CASH FLOWS
    # ============================================================
    
    print("STEP 3: Forecasting Free Cash Flows")
    print("-" * 80)
    
    # Use declining growth rates (more conservative)
    # Year 1-2: historical growth rate
    # Year 3-5: gradually decline to perpetual growth rate
    
    perpetual_growth = 0.025  # 2.5% (approximate GDP growth)
    print(f"Perpetual Growth Rate: {perpetual_growth*100:.2f}%\n")
    
    forecasted_fcf = []
    growth_rates = []
    
    for year in range(1, forecast_years + 1):
        if year <= 2:
            growth = fcf_growth_rate
        else:
            # Linear decline from historical to perpetual growth
            decline_factor = (year - 2) / (forecast_years - 2)
            growth = fcf_growth_rate * (1 - decline_factor) + perpetual_growth * decline_factor
        
        growth_rates.append(growth)
        
        if year == 1:
            fcf = ttm_fcf * (1 + growth)
        else:
            fcf = forecasted_fcf[-1] * (1 + growth)
        
        forecasted_fcf.append(fcf)
        print(f"Year {year}: FCF = ${fcf:,.0f} (Growth: {growth*100:.2f}%)")
    
    print()
    
    # ============================================================
    # STEP 4: CALCULATE PRESENT VALUE OF FCFs
    # ============================================================
    
    print("STEP 4: Discounting Cash Flows")
    print("-" * 80)
    
    pv_fcf = []
    for year, fcf in enumerate(forecasted_fcf, 1):
        pv = fcf / ((1 + wacc) ** year)
        pv_fcf.append(pv)
        print(f"Year {year}: PV = ${pv:,.0f}")
    
    sum_pv_fcf = sum(pv_fcf)
    print(f"\nSum of PV of Forecasted FCFs: ${sum_pv_fcf:,.0f}\n")
    
    # ============================================================
    # STEP 5: CALCULATE TERMINAL VALUE
    # ============================================================
    
    print("STEP 5: Terminal Value Calculation")
    print("-" * 80)
    
    terminal_fcf = forecasted_fcf[-1] * (1 + perpetual_growth)
    terminal_value = terminal_fcf / (wacc - perpetual_growth)
    pv_terminal_value = terminal_value / ((1 + wacc) ** forecast_years)
    
    print(f"Terminal Year FCF: ${terminal_fcf:,.0f}")
    print(f"Terminal Value: ${terminal_value:,.0f}")
    print(f"PV of Terminal Value: ${pv_terminal_value:,.0f}\n")
    
    # ============================================================
    # STEP 6: CALCULATE ENTERPRISE VALUE & EQUITY VALUE
    # ============================================================
    
    print("STEP 6: Enterprise & Equity Value")
    print("-" * 80)
    
    enterprise_value = sum_pv_fcf + pv_terminal_value
    equity_value = enterprise_value + cash - total_debt
    fair_value_per_share = equity_value / shares_outstanding
    
    print(f"Enterprise Value: ${enterprise_value:,.0f}")
    print(f"Plus: Cash: ${cash:,.0f}")
    print(f"Less: Debt: ${total_debt:,.0f}")
    print(f"Equity Value: ${equity_value:,.0f}")
    print(f"Fair Value per Share: ${fair_value_per_share:.2f}\n")
    
    # ============================================================
    # STEP 7: VALUATION SUMMARY
    # ============================================================
    
    print("="*80)
    print("VALUATION SUMMARY")
    print("="*80)
    print(f"Current Price: ${current_price:.2f}")
    print(f"Fair Value (DCF): ${fair_value_per_share:.2f}")
    
    upside = ((fair_value_per_share - current_price) / current_price) * 100
    print(f"Upside/Downside: {upside:+.2f}%")
    
    if upside > 20:
        recommendation = "UNDERVALUED - Consider Buying"
    elif upside > 0:
        recommendation = "FAIRLY VALUED - Hold"
    elif upside > -20:
        recommendation = "FAIRLY VALUED - Hold"
    else:
        recommendation = "OVERVALUED - Consider Selling"
    
    print(f"Recommendation: {recommendation}\n")
    
    # ============================================================
    # RETURN RESULTS
    # ============================================================
    
    results = {
        'symbol': symbol,
        'current_price': current_price,
        'fair_value': fair_value_per_share,
        'upside_pct': upside,
        'recommendation': recommendation,
        
        # Cash flow metrics
        'ttm_fcf': ttm_fcf,
        'fcf_growth_rate': fcf_growth_rate,
        'forecasted_fcf': forecasted_fcf,
        
        # WACC components
        'wacc': wacc,
        'cost_of_equity': cost_of_equity,
        'cost_of_debt': cost_of_debt,
        'tax_rate': tax_rate,
        'beta': beta,
        'risk_free_rate': risk_free_rate,
        
        # Valuation
        'enterprise_value': enterprise_value,
        'equity_value': equity_value,
        'terminal_value': terminal_value,
        'pv_terminal_value': pv_terminal_value,
        
        # Balance sheet
        'total_debt': total_debt,
        'cash': cash,
        'shares_outstanding': shares_outstanding
    }
    
    return results

# ============================================================
# SENSITIVITY ANALYSIS
# ============================================================

def sensitivity_analysis(symbol, wacc_range=0.02, growth_range=0.01):
    """
    Perform sensitivity analysis on WACC and perpetual growth rate
    """
    
    print("\n" + "="*80)
    print("SENSITIVITY ANALYSIS")
    print("="*80 + "\n")
    
    # Get base case
    base_results = calculate_dcf_valuation(symbol, forecast_years=5)
    base_wacc = base_results['wacc']
    base_growth = 0.025  # Perpetual growth
    base_fair_value = base_results['fair_value']
    
    # Create sensitivity table
    wacc_values = [base_wacc - wacc_range, base_wacc, base_wacc + wacc_range]
    growth_values = [base_growth - growth_range, base_growth, base_growth + growth_range]
    
    print("\nSensitivity Table: Fair Value per Share")
    print("-" * 80)
    print(f"{'WACC \\ Growth':<15}", end="")
    for g in growth_values:
        print(f"{g*100:>12.1f}%", end="")
    print()
    print("-" * 80)
    
    # Note: For true sensitivity, we'd need to re-run DCF with different parameters
    # This is a simplified version showing the concept
    for wacc in wacc_values:
        print(f"{wacc*100:>6.2f}%        ", end="")
        for growth in growth_values:
            # Approximate impact (simplified)
            wacc_impact = (base_wacc - wacc) / base_wacc
            growth_impact = (growth - base_growth) / (base_wacc - base_growth)
            adjusted_value = base_fair_value * (1 + wacc_impact * 0.5 + growth_impact * 0.3)
            print(f"${adjusted_value:>11.2f}", end="")
        print()
    
    print("\n" + "="*80 + "\n")

# ============================================================
# USAGE EXAMPLE
# ============================================================

if __name__ == "__main__":
    symbol = "GOOG"
    
    # Run DCF valuation
    results = calculate_dcf_valuation(symbol, forecast_years=5)
    
    # Optional: Run sensitivity analysis
    # sensitivity_analysis(symbol)
    
    # Export summary to CSV
    summary_df = pd.DataFrame([{
        'Symbol': results['symbol'],
        'Current Price': results['current_price'],
        'Fair Value': results['fair_value'],
        'Upside (%)': results['upside_pct'],
        'Recommendation': results['recommendation'],
        'WACC (%)': results['wacc'] * 100,
        'FCF Growth (%)': results['fcf_growth_rate'] * 100,
        'TTM FCF': results['ttm_fcf']
    }])
    
    summary_df.to_csv(f'{symbol}_dcf_summary.csv', index=False)
    print(f"✓ Summary exported to {symbol}_dcf_summary.csv")


DCF VALUATION FOR GOOG

Current Price: $338.53
Shares Outstanding: 5,407,000,000
Market Cap: $4,086,677,897,216

STEP 1: Historical Free Cash Flow Analysis
--------------------------------------------------------------------------------
TTM Operating Cash Flow: $151,424,000,000
TTM CapEx: $77,872,000,000
TTM Free Cash Flow: $73,552,000,000
Historical FCF Growth Rate: 10.00%
CapEx / Operating CF Ratio: 51.43%

STEP 2: Calculating WACC
--------------------------------------------------------------------------------
Total Debt: $33,713,000,000
Cash: $23,090,000,000
Cost of Debt: 1.66%
Effective Tax Rate: 18.23%
Risk-Free Rate (10Y Treasury): 4.24%
Market Risk Premium (Historical Avg): 6.00%
Beta: 1.09
Cost of Equity (CAPM): 10.76%

Weight of Equity: 99.18%
Weight of Debt: 0.82%
WACC: 10.68%

STEP 3: Forecasting Free Cash Flows
--------------------------------------------------------------------------------
Perpetual Growth Rate: 2.50%

Year 1: FCF = $80,907,200,000 (Growth: 10.00%)
Year 

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# PART 1: MANAGEMENT QUALITY ANALYSIS (QUANTITATIVE)
# ============================================================

def calculate_management_quality(symbol):
    """
    Quantitative assessment of management quality based on:
    - Capital allocation efficiency (ROIC trend)
    - Earnings quality (OCF/Net Income)
    - Cash conversion efficiency
    - Consistency of margins
    - Leverage management
    - Shareholder returns
    
    Returns: Score 0-100 and detailed metrics
    """
    
    print(f"\n{'='*80}")
    print(f"MANAGEMENT QUALITY ANALYSIS FOR {symbol}")
    print(f"{'='*80}\n")
    
    ticker = yf.Ticker(symbol)
    
    # Get financial statements
    cf_df = ticker.quarterly_cashflow.T.sort_index()
    bs_df = ticker.quarterly_balance_sheet.T.sort_index()
    is_df = ticker.quarterly_financials.T.sort_index()
    
    scores = {}
    metrics = {}
    
    # ============ 1. CAPITAL ALLOCATION EFFICIENCY (ROIC TREND) ============
    print("1. Capital Allocation Efficiency (ROIC Trend)")
    print("-" * 80)
    
    roic_list = []
    for i in range(min(8, len(is_df))):
        ebit = is_df.iloc[-(i+1)].get('EBIT')
        pretax = is_df.iloc[-(i+1)].get('Pretax Income')
        tax = is_df.iloc[-(i+1)].get('Tax Provision')
        
        total_debt = bs_df.iloc[-(i+1)].get('Total Debt', 0)
        cash = bs_df.iloc[-(i+1)].get('Cash And Cash Equivalents', 0)
        equity = bs_df.iloc[-(i+1)].get('Stockholders Equity', 0)
        
        if pd.notna(ebit) and pd.notna(pretax) and pretax > 0:
            tax_rate = tax / pretax if pd.notna(tax) else 0.21
            tax_rate = max(min(tax_rate, 0.35), 0.10)
            nopat = ebit * (1 - tax_rate)
            
            ic = (total_debt if pd.notna(total_debt) else 0) - (cash if pd.notna(cash) else 0) + (equity if pd.notna(equity) else 0)
            
            if ic > 0:
                roic = (nopat / ic) * 100
                roic_list.append(roic)
    
    if len(roic_list) >= 4:
        recent_roic = np.mean(roic_list[:4])
        older_roic = np.mean(roic_list[4:]) if len(roic_list) > 4 else recent_roic
        roic_trend = recent_roic - older_roic  # Positive = improving
        
        # Score: 0-20 based on ROIC level and trend
        roic_level_score = min(recent_roic / 20 * 10, 10)  # Max 10 points for ROIC
        roic_trend_score = max(min(roic_trend * 2 + 5, 10), 0)  # Max 10 points for trend
        scores['roic'] = roic_level_score + roic_trend_score
        
        metrics['recent_roic'] = recent_roic
        metrics['roic_trend'] = roic_trend
        
        print(f"Recent ROIC (avg last 4Q): {recent_roic:.2f}%")
        print(f"ROIC Trend: {roic_trend:+.2f}% ({'Improving' if roic_trend > 0 else 'Declining'})")
        print(f"Score: {scores['roic']:.1f}/20\n")
    else:
        scores['roic'] = 10  # Neutral
        print("Insufficient data for ROIC analysis\n")
    
    # ============ 2. EARNINGS QUALITY (OCF / Net Income) ============
    print("2. Earnings Quality (OCF / Net Income Ratio)")
    print("-" * 80)
    
    ocf_ni_ratios = []
    for i in range(min(8, len(cf_df))):
        ocf = cf_df.iloc[-(i+1)].get('Operating Cash Flow')
        ni = is_df.iloc[-(i+1)].get('Net Income')
        
        if pd.notna(ocf) and pd.notna(ni) and ni > 0:
            ratio = ocf / ni
            ocf_ni_ratios.append(ratio)
    
    if ocf_ni_ratios:
        avg_ratio = np.mean(ocf_ni_ratios)
        
        # Score: 0-15 (ratio > 1.2 = excellent, > 1.0 = good, < 0.8 = poor)
        if avg_ratio > 1.2:
            scores['earnings_quality'] = 15
        elif avg_ratio > 1.0:
            scores['earnings_quality'] = 12
        elif avg_ratio > 0.8:
            scores['earnings_quality'] = 8
        else:
            scores['earnings_quality'] = 4
        
        metrics['ocf_ni_ratio'] = avg_ratio
        
        quality = "Excellent" if avg_ratio > 1.2 else "Good" if avg_ratio > 1.0 else "Moderate" if avg_ratio > 0.8 else "Poor"
        print(f"Avg OCF/NI Ratio: {avg_ratio:.2f}x ({quality})")
        print(f"Score: {scores['earnings_quality']:.1f}/15\n")
    else:
        scores['earnings_quality'] = 7.5
        print("Insufficient data\n")
    
    # ============ 3. CASH CONVERSION EFFICIENCY ============
    print("3. Cash Conversion Efficiency (FCF / Net Income)")
    print("-" * 80)
    
    fcf_ni_ratios = []
    for i in range(min(8, len(cf_df))):
        ocf = cf_df.iloc[-(i+1)].get('Operating Cash Flow')
        capex = cf_df.iloc[-(i+1)].get('Capital Expenditure', 0)
        ni = is_df.iloc[-(i+1)].get('Net Income')
        
        if pd.notna(ocf) and pd.notna(ni) and ni > 0:
            fcf = ocf - abs(capex if pd.notna(capex) else 0)
            ratio = fcf / ni
            fcf_ni_ratios.append(ratio)
    
    if fcf_ni_ratios:
        avg_ratio = np.mean(fcf_ni_ratios)
        
        # Score: 0-15
        if avg_ratio > 1.0:
            scores['cash_conversion'] = 15
        elif avg_ratio > 0.7:
            scores['cash_conversion'] = 11
        elif avg_ratio > 0.4:
            scores['cash_conversion'] = 7
        else:
            scores['cash_conversion'] = 3
        
        metrics['fcf_ni_ratio'] = avg_ratio
        
        print(f"Avg FCF/NI Ratio: {avg_ratio:.2f}x")
        print(f"Score: {scores['cash_conversion']:.1f}/15\n")
    else:
        scores['cash_conversion'] = 7.5
        print("Insufficient data\n")
    
    # ============ 4. MARGIN CONSISTENCY ============
    print("4. Margin Consistency (Operating Margin Stability)")
    print("-" * 80)
    
    margins = []
    for i in range(min(8, len(is_df))):
        revenue = is_df.iloc[-(i+1)].get('Total Revenue')
        op_income = is_df.iloc[-(i+1)].get('Operating Income')
        
        if pd.notna(revenue) and pd.notna(op_income) and revenue > 0:
            margin = (op_income / revenue) * 100
            margins.append(margin)
    
    if len(margins) >= 4:
        std_dev = np.std(margins)
        avg_margin = np.mean(margins)
        
        # Score: 0-15 (lower std dev = more consistent = better)
        # CV (Coefficient of Variation) = std_dev / mean
        cv = std_dev / abs(avg_margin) if avg_margin != 0 else 1
        
        if cv < 0.1:  # Very consistent
            scores['consistency'] = 15
        elif cv < 0.2:
            scores['consistency'] = 11
        elif cv < 0.3:
            scores['consistency'] = 7
        else:
            scores['consistency'] = 3
        
        metrics['margin_std_dev'] = std_dev
        metrics['avg_margin'] = avg_margin
        
        print(f"Avg Operating Margin: {avg_margin:.2f}%")
        print(f"Std Dev: {std_dev:.2f}% (Coefficient of Variation: {cv:.2f})")
        print(f"Score: {scores['consistency']:.1f}/15\n")
    else:
        scores['consistency'] = 7.5
        print("Insufficient data\n")
    
    # ============ 5. LEVERAGE MANAGEMENT ============
    print("5. Leverage Management (Debt/Equity Trend)")
    print("-" * 80)
    
    de_ratios = []
    for i in range(min(8, len(bs_df))):
        debt = bs_df.iloc[-(i+1)].get('Total Debt', 0)
        equity = bs_df.iloc[-(i+1)].get('Stockholders Equity')
        
        if pd.notna(debt) and pd.notna(equity) and equity > 0:
            de_ratio = debt / equity
            de_ratios.append(de_ratio)
    
    if len(de_ratios) >= 4:
        recent_de = np.mean(de_ratios[:4])
        older_de = np.mean(de_ratios[4:]) if len(de_ratios) > 4 else recent_de
        de_trend = recent_de - older_de  # Negative = deleveraging = good
        
        # Score: 0-15 based on level and trend
        if recent_de < 0.5:  # Low leverage
            level_score = 10
        elif recent_de < 1.0:
            level_score = 7
        elif recent_de < 2.0:
            level_score = 4
        else:
            level_score = 1
        
        trend_score = 5 if de_trend < -0.1 else 3 if de_trend < 0.1 else 1
        
        scores['leverage'] = level_score + trend_score
        
        metrics['debt_equity_ratio'] = recent_de
        metrics['de_trend'] = de_trend
        
        print(f"Recent D/E Ratio: {recent_de:.2f}")
        print(f"Trend: {de_trend:+.2f} ({'Deleveraging' if de_trend < 0 else 'Increasing Leverage'})")
        print(f"Score: {scores['leverage']:.1f}/15\n")
    else:
        scores['leverage'] = 7.5
        print("Insufficient data\n")
    
    # ============ 6. SHAREHOLDER RETURNS ============
    print("6. Shareholder Returns (Dividends + Buybacks)")
    print("-" * 80)
    
    info = ticker.info
    dividend_yield = info.get('dividendYield', 0) or 0
    
    # Calculate buyback yield (approximate)
    shares_list = []
    for i in range(min(8, len(bs_df))):
        shares = info.get('sharesOutstanding')  # This is current, ideally need historical
        if shares:
            shares_list.append(shares)
    
    # Simplified: use dividend yield as proxy for total shareholder yield
    shareholder_yield = dividend_yield * 100
    
    # Score: 0-20
    if shareholder_yield > 3:
        scores['shareholder_returns'] = 20
    elif shareholder_yield > 2:
        scores['shareholder_returns'] = 15
    elif shareholder_yield > 1:
        scores['shareholder_returns'] = 10
    else:
        scores['shareholder_returns'] = 5
    
    metrics['dividend_yield'] = dividend_yield * 100
    
    print(f"Dividend Yield: {dividend_yield*100:.2f}%")
    print(f"Score: {scores['shareholder_returns']:.1f}/20\n")
    
    # ============ CALCULATE TOTAL SCORE ============
    total_score = sum(scores.values())
    
    print("="*80)
    print("MANAGEMENT QUALITY SCORE")
    print("="*80)
    print(f"Capital Allocation (ROIC): {scores.get('roic', 0):.1f}/20")
    print(f"Earnings Quality: {scores.get('earnings_quality', 0):.1f}/15")
    print(f"Cash Conversion: {scores.get('cash_conversion', 0):.1f}/15")
    print(f"Margin Consistency: {scores.get('consistency', 0):.1f}/15")
    print(f"Leverage Management: {scores.get('leverage', 0):.1f}/15")
    print(f"Shareholder Returns: {scores.get('shareholder_returns', 0):.1f}/20")
    print("-"*80)
    print(f"TOTAL SCORE: {total_score:.1f}/100")
    
    # Categorize
    if total_score >= 75:
        category = "EXCELLENT"
        quality_tier = "high"
    elif total_score >= 60:
        category = "GOOD"
        quality_tier = "medium-high"
    elif total_score >= 45:
        category = "AVERAGE"
        quality_tier = "medium"
    else:
        category = "BELOW AVERAGE"
        quality_tier = "low"
    
    print(f"Category: {category}")
    print(f"Quality Tier: {quality_tier.upper()}\n")
    
    return {
        'total_score': total_score,
        'category': category,
        'quality_tier': quality_tier,
        'scores': scores,
        'metrics': metrics
    }

def get_analyst_consensus(symbol):
    """
    Fetch analyst consensus data from Yahoo Finance and convert to scenario probabilities.
    Uses ONLY real data - no fake numbers.
    
    Returns: dict with probabilities and analyst data
    """
    
    print(f"\n{'='*80}")
    print(f"ANALYST CONSENSUS DATA FOR {symbol}")
    print(f"{'='*80}\n")
    
    ticker = yf.Ticker(symbol)
    info = ticker.info
    
    # Get available analyst data
    rec_mean = info.get('recommendationMean')  # 1.0-5.0 scale
    analyst_count = info.get('numberOfAnalystOpinions')
    rec_key = info.get('recommendationKey')  # e.g., 'buy', 'strong_buy'
    
    # Try to get detailed breakdown from recommendations history
    has_breakdown = False
    breakdown = {}
    
    try:
        recs = ticker.recommendations
        if recs is not None and not recs.empty:
            # Get most recent recommendations
            recent_recs = recs.tail(50)  # Last 50 recommendations
            
            # Count by grade
            if 'To Grade' in recent_recs.columns:
                grade_counts = recent_recs['To Grade'].value_counts()
                
                # Map various rating names to standard categories
                strong_buy_terms = ['Strong Buy', 'Outperform', 'Overweight']
                buy_terms = ['Buy', 'Positive']
                hold_terms = ['Hold', 'Neutral', 'Equal-Weight', 'Sector Perform', 'Market Perform']
                sell_terms = ['Underperform', 'Reduce', 'Underweight']
                strong_sell_terms = ['Strong Sell', 'Sell']
                
                breakdown['strong_buy'] = sum(grade_counts.get(term, 0) for term in strong_buy_terms)
                breakdown['buy'] = sum(grade_counts.get(term, 0) for term in buy_terms)
                breakdown['hold'] = sum(grade_counts.get(term, 0) for term in hold_terms)
                breakdown['sell'] = sum(grade_counts.get(term, 0) for term in sell_terms)
                breakdown['strong_sell'] = sum(grade_counts.get(term, 0) for term in strong_sell_terms)
                
                has_breakdown = True
                
                print("Recent Analyst Recommendations (Last 50):")
                print("-" * 80)
                for grade, count in grade_counts.head(10).items():
                    print(f"  {grade}: {count}")
                print()
    except Exception as e:
        print(f"Could not fetch detailed breakdown: {e}\n")
    
    # Display what we found
    print("Available Analyst Data:")
    print("-" * 80)
    
    has_data = False
    
    if analyst_count:
        print(f"Number of Analysts: {analyst_count}")
        has_data = True
    else:
        print("Number of Analysts: Not available")
    
    if rec_mean:
        print(f"Recommendation Mean: {rec_mean:.2f} (1.0=Strong Buy, 5.0=Sell)")
        has_data = True
    else:
        print("Recommendation Mean: Not available")
    
    if rec_key:
        print(f"Recommendation Key: {rec_key.upper()}")
        has_data = True
    else:
        print("Recommendation Key: Not available")
    
    print()
    
    # Convert to probabilities
    if rec_mean:
        # rec_mean scale: 1.0 = Strong Buy, 5.0 = Sell
        # Convert to probabilities
        
        if rec_mean <= 1.5:  # Strong Buy consensus
            prob_bull = 0.40
            prob_base = 0.45
            prob_bear = 0.15
            consensus_label = "STRONG BUY"
        elif rec_mean <= 2.0:  # Buy consensus
            prob_bull = 0.35
            prob_base = 0.50
            prob_bear = 0.15
            consensus_label = "BUY"
        elif rec_mean <= 2.5:  # Moderate Buy
            prob_bull = 0.30
            prob_base = 0.50
            prob_bear = 0.20
            consensus_label = "MODERATE BUY"
        elif rec_mean <= 3.0:  # Hold/Neutral
            prob_bull = 0.25
            prob_base = 0.50
            prob_bear = 0.25
            consensus_label = "HOLD"
        elif rec_mean <= 3.5:  # Moderate Sell
            prob_bull = 0.20
            prob_base = 0.50
            prob_bear = 0.30
            consensus_label = "MODERATE SELL"
        elif rec_mean <= 4.0:  # Sell
            prob_bull = 0.15
            prob_base = 0.45
            prob_bear = 0.40
            consensus_label = "SELL"
        else:  # Strong Sell
            prob_bull = 0.10
            prob_base = 0.40
            prob_bear = 0.50
            consensus_label = "STRONG SELL"
    
    elif has_breakdown and sum(breakdown.values()) > 0:
        # Use breakdown if available
        total = sum(breakdown.values())
        buy_pct = (breakdown.get('strong_buy', 0) + breakdown.get('buy', 0)) / total
        sell_pct = (breakdown.get('sell', 0) + breakdown.get('strong_sell', 0)) / total
        
        if buy_pct > 0.7:
            prob_bull = 0.35
            prob_base = 0.50
            prob_bear = 0.15
            consensus_label = "BULLISH"
        elif buy_pct > 0.5:
            prob_bull = 0.30
            prob_base = 0.50
            prob_bear = 0.20
            consensus_label = "MODERATELY BULLISH"
        elif sell_pct > 0.5:
            prob_bull = 0.20
            prob_base = 0.50
            prob_bear = 0.30
            consensus_label = "MODERATELY BEARISH"
        elif sell_pct > 0.3:
            prob_bull = 0.15
            prob_base = 0.45
            prob_bear = 0.40
            consensus_label = "BEARISH"
        else:
            prob_bull = 0.25
            prob_base = 0.50
            prob_bear = 0.25
            consensus_label = "NEUTRAL"
    
    else:
        # No analyst data - use neutral probabilities
        prob_bull = 0.25
        prob_base = 0.50
        prob_bear = 0.25
        consensus_label = "NO DATA - NEUTRAL ASSUMED"
    
    print(f"Consensus Interpretation: {consensus_label}")
    print(f"Derived Probabilities: Bull {prob_bull*100:.0f}%, Base {prob_base*100:.0f}%, Bear {prob_bear*100:.0f}%\n")
    
    return {
        'has_data': has_data,
        'has_breakdown': has_breakdown,
        'analyst_count': analyst_count,
        'rec_mean': rec_mean,
        'rec_key': rec_key,
        'consensus_label': consensus_label,
        'prob_bull': prob_bull,
        'prob_base': prob_base,
        'prob_bear': prob_bear,
        **breakdown
    }

# ============================================================
# PART 2: HISTORICAL MULTIPLES ANALYSIS
# ============================================================

def analyze_historical_multiples(symbol):
    """
    Analyze company's historical valuation multiples
    to determine if current valuation is high/low vs history
    """
    
    print(f"\n{'='*80}")
    print(f"HISTORICAL MULTIPLES ANALYSIS FOR {symbol}")
    print(f"{'='*80}\n")
    
    ticker = yf.Ticker(symbol)
    info = ticker.info
    
    # Get current multiples
    current_pe = info.get('trailingPE')
    current_pb = info.get('priceToBook')
    current_ps = info.get('priceToSalesTrailing12Months')
    
    # Get historical price data (5 years)
    hist = ticker.history(period="5y")
    
    # Get quarterly financials
    is_df = ticker.quarterly_financials.T.sort_index()
    bs_df = ticker.quarterly_balance_sheet.T.sort_index()
    
    print("Current Multiples:")
    print("-"*80)
    print(f"P/E Ratio: {current_pe:.2f}" if current_pe else "P/E Ratio: N/A")
    print(f"P/B Ratio: {current_pb:.2f}" if current_pb else "P/B Ratio: N/A")
    print(f"P/S Ratio: {current_ps:.2f}" if current_ps else "P/S Ratio: N/A")
    
    # Calculate historical ranges (simplified - would need historical P/E etc)
    # This is a placeholder for concept
    
    print("\n5-Year Historical Range (Estimated):")
    print("-"*80)
    print("P/E: 15-35 (Current in middle range)")
    print("P/B: 3-8 (Current in upper range)")
    print("P/S: 4-9 (Current in middle range)")
    print("\nNote: Use actual historical data for precise analysis\n")
    
    # Determine valuation level
    valuation_signal = "neutral"  # Placeholder
    
    return {
        'current_pe': current_pe,
        'current_pb': current_pb,
        'current_ps': current_ps,
        'valuation_signal': valuation_signal
    }

# ============================================================
# PART 3: SCENARIO-BASED DCF MODEL
# ============================================================

def get_base_case_inputs(symbol):
    """
    Get base case inputs from historical data
    """
    ticker = yf.Ticker(symbol)
    
    # Get financial statements
    cf_df = ticker.quarterly_cashflow.T.sort_index()
    bs_df = ticker.quarterly_balance_sheet.T.sort_index()
    is_df = ticker.quarterly_financials.T.sort_index()
    
    # Get market data
    info = ticker.info
    current_price = info.get('currentPrice', info.get('regularMarketPrice'))
    shares_outstanding = info.get('sharesOutstanding')
    market_cap = info.get('marketCap')
    beta = info.get('beta', 1.0)
    
    # Calculate TTM FCF
    operating_cf_list = []
    capex_list = []
    
    for i in range(min(4, len(cf_df))):
        ocf = cf_df.iloc[-(i+1)].get('Operating Cash Flow')
        capex = cf_df.iloc[-(i+1)].get('Capital Expenditure', 0)
        
        if pd.notna(ocf):
            operating_cf_list.append(ocf)
            capex_list.append(abs(capex) if pd.notna(capex) else 0)
    
    ttm_operating_cf = sum(operating_cf_list)
    ttm_capex = sum(capex_list)
    ttm_fcf = ttm_operating_cf - ttm_capex
    
    # Calculate historical FCF growth
    fcf_list = []
    for i in range(min(8, len(cf_df))):
        ocf = cf_df.iloc[-(i+1)].get('Operating Cash Flow')
        capex = cf_df.iloc[-(i+1)].get('Capital Expenditure', 0)
        if pd.notna(ocf):
            fcf_list.append(ocf - abs(capex if pd.notna(capex) else 0))
    
    if len(fcf_list) >= 8:
        recent_fcf = sum(fcf_list[:4])
        previous_fcf = sum(fcf_list[4:8])
        fcf_growth_rate = (recent_fcf / previous_fcf - 1) if previous_fcf > 0 else 0.10
        fcf_growth_rate = max(min(fcf_growth_rate, 0.30), -0.10)
    else:
        fcf_growth_rate = 0.10
    
    # Calculate tax rate
    tax_list = []
    pretax_list = []
    for i in range(min(4, len(is_df))):
        tax = is_df.iloc[-(i+1)].get('Tax Provision')
        pretax = is_df.iloc[-(i+1)].get('Pretax Income')
        if pd.notna(tax) and pd.notna(pretax) and pretax > 0:
            tax_list.append(tax)
            pretax_list.append(pretax)
    
    tax_rate = sum(tax_list) / sum(pretax_list) if pretax_list else 0.21
    tax_rate = max(min(tax_rate, 0.35), 0.10)
    
    # Get debt and cash
    total_debt = bs_df.iloc[-1].get('Total Debt', 0)
    if pd.isna(total_debt):
        total_debt = (bs_df.iloc[-1].get('Long Term Debt', 0) or 0) + (bs_df.iloc[-1].get('Current Debt', 0) or 0)
    
    cash = bs_df.iloc[-1].get('Cash And Cash Equivalents', 0)
    if pd.isna(cash):
        cash = bs_df.iloc[-1].get('Cash Cash Equivalents And Short Term Investments', 0) or 0
    
    # Get risk-free rate
    try:
        tnx = yf.Ticker("^TNX")
        risk_free_rate = tnx.info.get('regularMarketPrice', 4.5) / 100
    except:
        risk_free_rate = 0.045
    
    # Calculate cost of debt
    interest_list = []
    debt_list = []
    for i in range(min(4, len(is_df))):
        interest = is_df.iloc[-(i+1)].get('Interest Expense')
        debt = bs_df.iloc[-(i+1)].get('Total Debt')
        if pd.notna(interest) and pd.notna(debt) and debt > 0:
            interest_list.append(abs(interest))
            debt_list.append(debt)
    
    cost_of_debt = sum(interest_list) / np.mean(debt_list) if debt_list else 0.04
    
    return {
        'ttm_fcf': ttm_fcf,
        'fcf_growth_rate': fcf_growth_rate,
        'beta': beta,
        'tax_rate': tax_rate,
        'total_debt': total_debt,
        'cash': cash,
        'shares_outstanding': shares_outstanding,
        'market_cap': market_cap,
        'current_price': current_price,
        'risk_free_rate': risk_free_rate,
        'cost_of_debt': cost_of_debt
    }

def scenario_dcf(symbol, scenario_type, base_inputs, mgmt_quality_tier, forecast_years=5):
    """
    Run DCF for a specific scenario (bull/base/bear)
    
    scenario_type: 'bull', 'base', or 'bear'
    mgmt_quality_tier: 'high', 'medium-high', 'medium', or 'low'
    
    Management quality affects performance WITHIN each scenario:
    - Good mgmt: Better upside capture in bull, less downside in bear
    - Poor mgmt: Misses opportunities in bull, worse damage in bear
    """
    
    # Extract base inputs
    ttm_fcf = base_inputs['ttm_fcf']
    base_fcf_growth = base_inputs['fcf_growth_rate']
    base_beta = base_inputs['beta']
    tax_rate = base_inputs['tax_rate']
    total_debt = base_inputs['total_debt']
    cash = base_inputs['cash']
    shares = base_inputs['shares_outstanding']
    market_cap = base_inputs['market_cap']
    risk_free_rate = base_inputs['risk_free_rate']
    cost_of_debt = base_inputs['cost_of_debt']
    
    # Define BASE scenario parameters (external environment)
    if scenario_type == 'bull':
        base_growth_early = base_fcf_growth + 0.10  # +10% for bull market
        base_growth_late = base_fcf_growth + 0.07   # +7%
        base_terminal_growth = 0.030  # 3%
        base_beta_adj = -0.2  # Lower risk in bull market
    elif scenario_type == 'base':
        base_growth_early = base_fcf_growth
        base_growth_late = base_fcf_growth * 0.7
        base_terminal_growth = 0.025  # 2.5%
        base_beta_adj = 0
    else:  # bear
        base_growth_early = max(base_fcf_growth - 0.05, 0.01)  # -5%, min 1%
        base_growth_late = max(base_fcf_growth - 0.08, 0.005)   # -8%, min 0.5%
        base_terminal_growth = 0.020  # 2%
        base_beta_adj = 0.3  # Higher risk in bear market
    
    # ADJUST based on management quality
    # High quality = better execution, lower downside risk
    # Low quality = poor execution, higher downside risk
    
    if mgmt_quality_tier == 'high':
        if scenario_type == 'bull':
            mgmt_adj_early = 0.03  # Captures 3% more upside
            mgmt_adj_late = 0.02
            mgmt_beta_adj = -0.05  # Better execution reduces risk
        elif scenario_type == 'base':
            mgmt_adj_early = 0.02  # Slight outperformance
            mgmt_adj_late = 0.01
            mgmt_beta_adj = -0.03
        else:  # bear
            mgmt_adj_early = 0.03  # Defensive - limits damage
            mgmt_adj_late = 0.02
            mgmt_beta_adj = -0.1  # Better crisis management
    
    elif mgmt_quality_tier == 'medium-high':
        if scenario_type == 'bull':
            mgmt_adj_early = 0.02
            mgmt_adj_late = 0.01
            mgmt_beta_adj = -0.03
        elif scenario_type == 'base':
            mgmt_adj_early = 0.01
            mgmt_adj_late = 0.005
            mgmt_beta_adj = -0.01
        else:  # bear
            mgmt_adj_early = 0.02
            mgmt_adj_late = 0.01
            mgmt_beta_adj = -0.05
    
    elif mgmt_quality_tier == 'medium':
        # Average management = no adjustment
        mgmt_adj_early = 0
        mgmt_adj_late = 0
        mgmt_beta_adj = 0
    
    else:  # low quality
        if scenario_type == 'bull':
            mgmt_adj_early = -0.03  # Misses opportunities
            mgmt_adj_late = -0.02
            mgmt_beta_adj = 0.05  # Poor execution increases risk
        elif scenario_type == 'base':
            mgmt_adj_early = -0.02  # Underperformance
            mgmt_adj_late = -0.01
            mgmt_beta_adj = 0.03
        else:  # bear
            mgmt_adj_early = -0.04  # Severe damage from poor crisis mgmt
            mgmt_adj_late = -0.03
            mgmt_beta_adj = 0.15  # Much higher risk
    
    # Apply adjustments
    fcf_growth_early = base_growth_early + mgmt_adj_early
    fcf_growth_late = base_growth_late + mgmt_adj_late
    terminal_growth = base_terminal_growth
    beta_adj = base_beta_adj + mgmt_beta_adj
    
    # Calculate WACC
    adjusted_beta = max(base_beta + beta_adj, 0.5)  # Min beta of 0.5
    market_risk_premium = 0.06
    cost_of_equity = risk_free_rate + (adjusted_beta * market_risk_premium)
    
    total_value = market_cap + total_debt
    weight_equity = market_cap / total_value if total_value > 0 else 1
    weight_debt = total_debt / total_value if total_value > 0 else 0
    
    wacc = (weight_equity * cost_of_equity) + (weight_debt * cost_of_debt * (1 - tax_rate))
    
    # Forecast FCF
    forecasted_fcf = []
    for year in range(1, forecast_years + 1):
        if year <= 3:
            growth = fcf_growth_early
        else:
            growth = fcf_growth_late
        
        if year == 1:
            fcf = ttm_fcf * (1 + growth)
        else:
            fcf = forecasted_fcf[-1] * (1 + growth)
        
        forecasted_fcf.append(fcf)
    
    # Discount FCF
    pv_fcf = []
    for year, fcf in enumerate(forecasted_fcf, 1):
        pv = fcf / ((1 + wacc) ** year)
        pv_fcf.append(pv)
    
    sum_pv_fcf = sum(pv_fcf)
    
    # Terminal Value
    terminal_fcf = forecasted_fcf[-1] * (1 + terminal_growth)
    terminal_value = terminal_fcf / (wacc - terminal_growth)
    pv_terminal_value = terminal_value / ((1 + wacc) ** forecast_years)
    
    # Enterprise and Equity Value
    enterprise_value = sum_pv_fcf + pv_terminal_value
    equity_value = enterprise_value + cash - total_debt
    fair_value_per_share = equity_value / shares
    
    return {
        'scenario': scenario_type,
        'fair_value': fair_value_per_share,
        'enterprise_value': enterprise_value,
        'equity_value': equity_value,
        'wacc': wacc,
        'terminal_growth': terminal_growth,
        'fcf_growth_early': fcf_growth_early,
        'fcf_growth_late': fcf_growth_late,
        'beta': adjusted_beta,
        'forecasted_fcf': forecasted_fcf,
        'pv_terminal_value': pv_terminal_value,
        'mgmt_adjustment': f"Early: {mgmt_adj_early:+.1%}, Late: {mgmt_adj_late:+.1%}"
    }

# ============================================================
# PART 4: PROBABILITY-WEIGHTED VALUATION
# ============================================================

def probability_weighted_valuation(symbol):
    """
    Complete valuation framework:
    1. Assess management quality
    2. Analyze historical multiples
    3. Run scenario DCF (bull/base/bear)
    4. Adjust probabilities based on management quality
    5. Calculate probability-weighted fair value
    """
    
    print(f"\n{'='*80}")
    print(f"COMPREHENSIVE VALUATION ANALYSIS FOR {symbol}")
    print(f"{'='*80}\n")
    
    # Step 1: Management Quality
    mgmt_analysis = calculate_management_quality(symbol)
    mgmt_score = mgmt_analysis['total_score']
    quality_tier = mgmt_analysis['quality_tier']
    
    # Step 2: Historical Multiples
    multiples = analyze_historical_multiples(symbol)
    
    # Step 3: Get base case inputs
    base_inputs = get_base_case_inputs(symbol)
    current_price = base_inputs['current_price']
    
    # Step 4: Get analyst consensus for probabilities
    analyst_data = get_analyst_consensus(symbol)
    
    # Set probabilities based on analyst consensus
    prob_bull = analyst_data['prob_bull']
    prob_base = analyst_data['prob_base']
    prob_bear = analyst_data['prob_bear']
    
    print(f"\n{'='*80}")
    print("SCENARIO PROBABILITIES (Based on Analyst Consensus)")
    print(f"{'='*80}")
    
    if analyst_data['has_data']:
        print(f"Analyst Coverage: {analyst_data['analyst_count']} analysts")
        print(f"Recommendation Mean: {analyst_data['rec_mean']:.2f} (1=Strong Buy, 5=Sell)")
        print(f"Consensus: {analyst_data['consensus_label']}")
        
        if analyst_data['has_breakdown']:
            print(f"\nRecommendation Breakdown:")
            print(f"  Strong Buy: {analyst_data.get('strong_buy', 'N/A')}")
            print(f"  Buy: {analyst_data.get('buy', 'N/A')}")
            print(f"  Hold: {analyst_data.get('hold', 'N/A')}")
            print(f"  Sell: {analyst_data.get('sell', 'N/A')}")
            print(f"  Strong Sell: {analyst_data.get('strong_sell', 'N/A')}")
    else:
        print("No analyst data available - using neutral probabilities")
    
    print(f"\nScenario Probabilities:")
    print(f"  Bull Case: {prob_bull*100:.0f}%")
    print(f"  Base Case: {prob_base*100:.0f}%")
    print(f"  Bear Case: {prob_bear*100:.0f}%\n")
    
    # Step 5: Run scenario DCF with management quality adjustments
    print(f"{'='*80}")
    print("RUNNING SCENARIO DCF MODELS")
    print(f"{'='*80}")
    print(f"Management Quality: {mgmt_analysis['category']} ({quality_tier.upper()})")
    print("(Quality affects performance within each scenario)\n")
    
    bull_result = scenario_dcf(symbol, 'bull', base_inputs, quality_tier)
    base_result = scenario_dcf(symbol, 'base', base_inputs, quality_tier)
    bear_result = scenario_dcf(symbol, 'bear', base_inputs, quality_tier)
    
    print("BULL CASE SCENARIO")
    print("-"*80)
    print(f"FCF Growth (Early Years): {bull_result['fcf_growth_early']*100:.1f}%")
    print(f"FCF Growth (Later Years): {bull_result['fcf_growth_late']*100:.1f}%")
    print(f"Management Adjustment: {bull_result['mgmt_adjustment']}")
    print(f"Terminal Growth: {bull_result['terminal_growth']*100:.1f}%")
    print(f"WACC: {bull_result['wacc']*100:.2f}%")
    print(f"Beta: {bull_result['beta']:.2f}")
    print(f"Fair Value: ${bull_result['fair_value']:.2f}\n")
    
    print("BASE CASE SCENARIO")
    print("-"*80)
    print(f"FCF Growth (Early Years): {base_result['fcf_growth_early']*100:.1f}%")
    print(f"FCF Growth (Later Years): {base_result['fcf_growth_late']*100:.1f}%")
    print(f"Management Adjustment: {base_result['mgmt_adjustment']}")
    print(f"Terminal Growth: {base_result['terminal_growth']*100:.1f}%")
    print(f"WACC: {base_result['wacc']*100:.2f}%")
    print(f"Beta: {base_result['beta']:.2f}")
    print(f"Fair Value: ${base_result['fair_value']:.2f}\n")
    
    print("BEAR CASE SCENARIO")
    print("-"*80)
    print(f"FCF Growth (Early Years): {bear_result['fcf_growth_early']*100:.1f}%")
    print(f"FCF Growth (Later Years): {bear_result['fcf_growth_late']*100:.1f}%")
    print(f"Management Adjustment: {bear_result['mgmt_adjustment']}")
    print(f"Terminal Growth: {bear_result['terminal_growth']*100:.1f}%")
    print(f"WACC: {bear_result['wacc']*100:.2f}%")
    print(f"Beta: {bear_result['beta']:.2f}")
    print(f"Fair Value: ${bear_result['fair_value']:.2f}\n")
    
    # Step 6: Calculate probability-weighted fair value
    weighted_fair_value = (
        bull_result['fair_value'] * prob_bull +
        base_result['fair_value'] * prob_base +
        bear_result['fair_value'] * prob_bear
    )
    
    upside = ((weighted_fair_value - current_price) / current_price) * 100
    
    print(f"{'='*80}")
    print("FINAL VALUATION SUMMARY")
    print(f"{'='*80}")
    print(f"Current Price: ${current_price:.2f}")
    print(f"\nScenario Fair Values:")
    print(f"  Bull Case ({prob_bull*100:.0f}%): ${bull_result['fair_value']:.2f}")
    print(f"  Base Case ({prob_base*100:.0f}%): ${base_result['fair_value']:.2f}")
    print(f"  Bear Case ({prob_bear*100:.0f}%): ${bear_result['fair_value']:.2f}")
    print(f"\nProbability-Weighted Fair Value: ${weighted_fair_value:.2f}")
    print(f"Upside/Downside: {upside:+.2f}%")
    
    # Investment recommendation
    if upside > 25:
        recommendation = "STRONG BUY"
    elif upside > 10:
        recommendation = "BUY"
    elif upside > -10:
        recommendation = "HOLD"
    elif upside > -25:
        recommendation = "SELL"
    else:
        recommendation = "STRONG SELL"
    
    print(f"\nInvestment Recommendation: {recommendation}")
    
    # Risk assessment
    value_range = bull_result['fair_value'] - bear_result['fair_value']
    risk_level = "High" if value_range / weighted_fair_value > 0.5 else "Moderate" if value_range / weighted_fair_value > 0.3 else "Low"
    
    print(f"Risk Level: {risk_level}")
    print(f"Value Range: ${bear_result['fair_value']:.2f} - ${bull_result['fair_value']:.2f}")
    print(f"{'='*80}\n")
    
    # Create summary DataFrame
    summary_data = {
        'Metric': [
            'Current Price',
            'Bull Case Fair Value',
            'Base Case Fair Value',
            'Bear Case Fair Value',
            'Weighted Fair Value',
            'Upside/Downside (%)',
            'Management Quality Score',
            'Recommendation',
            'Risk Level'
        ],
        'Value': [
            f"${current_price:.2f}",
            f"${bull_result['fair_value']:.2f}",
            f"${base_result['fair_value']:.2f}",
            f"${bear_result['fair_value']:.2f}",
            f"${weighted_fair_value:.2f}",
            f"{upside:+.2f}%",
            f"{mgmt_score:.1f}/100 ({quality_tier.upper()})",
            recommendation,
            risk_level
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    
    # Export to CSV
    summary_df.to_csv(f'{symbol}_valuation_summary.csv', index=False)
    print(f"✓ Valuation summary exported to {symbol}_valuation_summary.csv\n")
    
    return {
        'symbol': symbol,
        'current_price': current_price,
        'weighted_fair_value': weighted_fair_value,
        'upside_pct': upside,
        'recommendation': recommendation,
        'risk_level': risk_level,
        'management_quality': mgmt_analysis,
        'bull_case': bull_result,
        'base_case': base_result,
        'bear_case': bear_result,
        'probabilities': {
            'bull': prob_bull,
            'base': prob_base,
            'bear': prob_bear
        }
    }

# ============================================================
# USAGE EXAMPLE
# ============================================================

if __name__ == "__main__":
    symbol = "GOOG"
    
    # Run comprehensive valuation
    results = probability_weighted_valuation(symbol)
    
    print("\n" + "="*80)
    print("ANALYSIS COMPLETE")
    print("="*80)
    print(f"\nKey Takeaways:")
    print(f"- Management Quality: {results['management_quality']['category']}")
    print(f"- Weighted Fair Value: ${results['weighted_fair_value']:.2f}")
    print(f"- Expected Return: {results['upside_pct']:+.1f}%")
    print(f"- Recommendation: {results['recommendation']}")
    print(f"- Risk Level: {results['risk_level']}\n")


COMPREHENSIVE VALUATION ANALYSIS FOR GOOG


MANAGEMENT QUALITY ANALYSIS FOR GOOG

1. Capital Allocation Efficiency (ROIC Trend)
--------------------------------------------------------------------------------
Recent ROIC (avg last 4Q): 8.62%
ROIC Trend: +0.42% (Improving)
Score: 10.1/20

2. Earnings Quality (OCF / Net Income Ratio)
--------------------------------------------------------------------------------
Avg OCF/NI Ratio: 1.21x (Excellent)
Score: 15.0/15

3. Cash Conversion Efficiency (FCF / Net Income)
--------------------------------------------------------------------------------
Avg FCF/NI Ratio: 0.61x
Score: 7.0/15

4. Margin Consistency (Operating Margin Stability)
--------------------------------------------------------------------------------
Avg Operating Margin: 32.26%
Std Dev: 1.08% (Coefficient of Variation: 0.03)
Score: 15.0/15

5. Leverage Management (Debt/Equity Trend)
--------------------------------------------------------------------------------
Recent D/E Rat

In [17]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# PART 1: SMART PEER SELECTION
# ============================================================

def get_peer_companies(symbol, sector=None, max_peers=15):
    """
    Intelligently select peer companies based on:
    - Same sector/industry
    - Similar market cap (within 0.3x - 3x range)
    - Similar growth profile
    - Excludes target company
    
    Returns: List of peer ticker symbols
    """
    
    print(f"\n{'='*80}")
    print(f"SELECTING PEER COMPANIES FOR {symbol}")
    print(f"{'='*80}\n")
    
    ticker = yf.Ticker(symbol)
    info = ticker.info
    
    # Get target company info
    target_sector = info.get('sector', sector)
    target_industry = info.get('industry')
    target_market_cap = info.get('marketCap', 0)
    
    print(f"Target Company: {symbol}")
    print(f"Sector: {target_sector}")
    print(f"Industry: {target_industry}")
    print(f"Market Cap: ${target_market_cap:,.0f}\n")
    
    # Define peer candidates by sector
    sector_peers = {
        'Technology': ['AAPL', 'MSFT', 'GOOGL', 'META', 'NVDA', 'ORCL', 'CSCO', 'ADBE', 'CRM', 'INTC', 
                       'AMD', 'QCOM', 'TXN', 'AVGO', 'NOW', 'INTU', 'IBM', 'SNOW', 'PANW', 'CRWD'],
        'Communication Services': ['GOOGL', 'META', 'NFLX', 'DIS', 'CMCSA', 'T', 'VZ', 'TMUS', 'CHTR'],
        'Consumer Cyclical': ['AMZN', 'TSLA', 'HD', 'MCD', 'NKE', 'SBUX', 'TGT', 'LOW', 'TJX', 'BKNG'],
        'Consumer Defensive': ['WMT', 'PG', 'KO', 'PEP', 'COST', 'PM', 'MO', 'MDLZ', 'CL', 'KHC'],
        'Healthcare': ['UNH', 'JNJ', 'LLY', 'ABBV', 'MRK', 'PFE', 'TMO', 'ABT', 'DHR', 'CVS', 'AMGN'],
        'Financial Services': ['BRK-B', 'JPM', 'V', 'MA', 'BAC', 'WFC', 'MS', 'GS', 'SPGI', 'BLK', 'C'],
        'Industrials': ['BA', 'UNP', 'HON', 'UPS', 'RTX', 'CAT', 'LMT', 'DE', 'GE', 'MMM'],
        'Energy': ['XOM', 'CVX', 'COP', 'SLB', 'EOG', 'MPC', 'PSX', 'VLO', 'OXY', 'HAL'],
        'Basic Materials': ['LIN', 'APD', 'SHW', 'ECL', 'DD', 'NEM', 'FCX', 'NUE'],
        'Real Estate': ['AMT', 'PLD', 'CCI', 'EQIX', 'PSA', 'SPG', 'WELL', 'DLR', 'O', 'VICI'],
        'Utilities': ['NEE', 'DUK', 'SO', 'D', 'AEP', 'EXC', 'SRE', 'PCG', 'XEL']
    }
    
    # Get candidate list
    candidates = sector_peers.get(target_sector, [])
    
    if not candidates:
        print(f"Warning: No predefined peers for sector '{target_sector}'")
        print("Using manual peer list or defaulting to similar companies\n")
        return []
    
    print(f"Screening {len(candidates)} candidates in {target_sector} sector...")
    
    # Filter peers based on criteria
    valid_peers = []
    peer_data = []
    
    for peer_symbol in candidates:
        if peer_symbol == symbol:
            continue
            
        try:
            peer = yf.Ticker(peer_symbol)
            peer_info = peer.info
            
            peer_mc = peer_info.get('marketCap', 0)
            
            # Skip if no market cap data
            if not peer_mc or peer_mc == 0:
                continue
            
            # RELAXED Market cap filter: within 0.1x to 10x of target (was 0.3x to 3x)
            mc_ratio = peer_mc / target_market_cap if target_market_cap > 0 else 0
            
            if 0.1 <= mc_ratio <= 10.0:
                valid_peers.append(peer_symbol)
                peer_data.append({
                    'symbol': peer_symbol,
                    'market_cap': peer_mc,
                    'mc_ratio': mc_ratio
                })
        
        except Exception as e:
            continue
    
    # Sort by market cap similarity and take top N
    peer_data_df = pd.DataFrame(peer_data)
    if not peer_data_df.empty:
        peer_data_df['mc_distance'] = abs(peer_data_df['mc_ratio'] - 1.0)
        peer_data_df = peer_data_df.sort_values('mc_distance').head(max_peers)
        valid_peers = peer_data_df['symbol'].tolist()
    
    print(f"\nSelected {len(valid_peers)} peer companies:")
    print("-" * 80)
    for peer_symbol in valid_peers:
        peer_info = next(p for p in peer_data if p['symbol'] == peer_symbol)
        print(f"  {peer_symbol}: Market Cap ${peer_info['market_cap']:,.0f} ({peer_info['mc_ratio']:.2f}x)")
    print()
    
    return valid_peers

# ============================================================
# PART 2: COLLECT PEER FINANCIAL DATA
# ============================================================

def collect_peer_data(symbol, peer_list):
    """
    Collect financial metrics for target company and peers
    
    Returns: DataFrame with all companies and their metrics
    """
    
    print(f"\n{'='*80}")
    print("COLLECTING PEER FINANCIAL DATA")
    print(f"{'='*80}\n")
    
    all_companies = [symbol] + peer_list
    data_list = []
    
    for ticker_symbol in all_companies:
        try:
            print(f"Fetching data for {ticker_symbol}...")
            ticker = yf.Ticker(ticker_symbol)
            info = ticker.info
            
            # Get financial statements
            try:
                cf_df = ticker.quarterly_cashflow.T.sort_index()
                bs_df = ticker.quarterly_balance_sheet.T.sort_index()
                is_df = ticker.quarterly_financials.T.sort_index()
            except:
                print(f"  Warning: Could not fetch financial statements for {ticker_symbol}")
                continue
            
            # Calculate TTM metrics
            revenue_list = []
            for i in range(min(4, len(is_df))):
                rev = is_df.iloc[-(i+1)].get('Total Revenue')
                if pd.notna(rev):
                    revenue_list.append(rev)
            ttm_revenue = sum(revenue_list) if revenue_list else None
            
            ni_list = []
            for i in range(min(4, len(is_df))):
                ni = is_df.iloc[-(i+1)].get('Net Income')
                if pd.notna(ni):
                    ni_list.append(ni)
            ttm_net_income = sum(ni_list) if ni_list else None
            
            ebitda_list = []
            for i in range(min(4, len(is_df))):
                ebitda = is_df.iloc[-(i+1)].get('EBITDA')
                if pd.notna(ebitda):
                    ebitda_list.append(ebitda)
            ttm_ebitda = sum(ebitda_list) if ebitda_list else None
            
            fcf_list = []
            for i in range(min(4, len(cf_df))):
                ocf = cf_df.iloc[-(i+1)].get('Operating Cash Flow')
                capex = cf_df.iloc[-(i+1)].get('Capital Expenditure', 0)
                if pd.notna(ocf):
                    fcf = ocf - abs(capex if pd.notna(capex) else 0)
                    fcf_list.append(fcf)
            ttm_fcf = sum(fcf_list) if fcf_list else None
            
            # Balance Sheet metrics
            total_equity = bs_df.iloc[-1].get('Stockholders Equity')
            total_assets = bs_df.iloc[-1].get('Total Assets')
            total_debt = bs_df.iloc[-1].get('Total Debt', 0)
            cash = bs_df.iloc[-1].get('Cash And Cash Equivalents', 0)
            
            # Calculate growth rates (YoY)
            revenue_growth = None
            if len(revenue_list) >= 4:
                recent_rev = sum(revenue_list[:4])
                try:
                    old_revenue_list = []
                    for i in range(4, min(8, len(is_df))):
                        old_rev = is_df.iloc[-(i+1)].get('Total Revenue')
                        if pd.notna(old_rev):
                            old_revenue_list.append(old_rev)
                    
                    if old_revenue_list:
                        old_rev = sum(old_revenue_list[:4])
                        revenue_growth = ((recent_rev - old_rev) / old_rev * 100) if old_rev else None
                except:
                    pass
            
            # Get market data
            market_cap = info.get('marketCap')
            current_price = info.get('currentPrice') or info.get('regularMarketPrice')
            shares_outstanding = info.get('sharesOutstanding')
            beta = info.get('beta')
            
            # Calculate valuation ratios
            pe_ratio = info.get('trailingPE')
            pb_ratio = market_cap / total_equity if (market_cap and total_equity and total_equity > 0) else None
            ps_ratio = market_cap / ttm_revenue if (market_cap and ttm_revenue and ttm_revenue > 0) else None
            
            # EV/EBITDA
            enterprise_value = market_cap + total_debt - cash if market_cap else None
            ev_ebitda = enterprise_value / ttm_ebitda if (enterprise_value and ttm_ebitda and ttm_ebitda > 0) else None
            
            # PEG ratio
            peg_ratio = pe_ratio / revenue_growth if (pe_ratio and revenue_growth and revenue_growth > 0) else None
            
            # Profitability metrics
            net_margin = (ttm_net_income / ttm_revenue * 100) if (ttm_net_income and ttm_revenue and ttm_revenue > 0) else None
            ebitda_margin = (ttm_ebitda / ttm_revenue * 100) if (ttm_ebitda and ttm_revenue and ttm_revenue > 0) else None
            roe = (ttm_net_income / total_equity * 100) if (ttm_net_income and total_equity and total_equity > 0) else None
            roa = (ttm_net_income / total_assets * 100) if (ttm_net_income and total_assets and total_assets > 0) else None
            
            # FCF Yield
            fcf_yield = (ttm_fcf / market_cap * 100) if (ttm_fcf and market_cap and market_cap > 0) else None
            
            # Compile data
            company_data = {
                'Symbol': ticker_symbol,
                'Market Cap': market_cap,
                'Price': current_price,
                'Revenue (TTM)': ttm_revenue,
                'Net Income (TTM)': ttm_net_income,
                'EBITDA (TTM)': ttm_ebitda,
                'FCF (TTM)': ttm_fcf,
                'Total Equity': total_equity,
                'Total Debt': total_debt,
                'Beta': beta,
                'P/E': pe_ratio,
                'P/B': pb_ratio,
                'P/S': ps_ratio,
                'EV/EBITDA': ev_ebitda,
                'PEG': peg_ratio,
                'Revenue Growth (%)': revenue_growth,
                'Net Margin (%)': net_margin,
                'EBITDA Margin (%)': ebitda_margin,
                'ROE (%)': roe,
                'ROA (%)': roa,
                'FCF Yield (%)': fcf_yield
            }
            
            data_list.append(company_data)
            
        except Exception as e:
            print(f"  Error processing {ticker_symbol}: {str(e)}")
            continue
    
    peer_df = pd.DataFrame(data_list)
    
    print(f"\n✓ Successfully collected data for {len(peer_df)} companies\n")
    
    return peer_df

# ============================================================
# PART 3: PERCENTILE RANKINGS
# ============================================================

def calculate_percentile_rankings(peer_df, target_symbol):
    """
    Calculate where the target company ranks vs peers on key metrics
    """
    
    print(f"\n{'='*80}")
    print(f"PERCENTILE RANKINGS FOR {target_symbol}")
    print(f"{'='*80}\n")
    
    target_data = peer_df[peer_df['Symbol'] == target_symbol].iloc[0]
    
    # Metrics where HIGHER is better
    higher_is_better = ['Revenue Growth (%)', 'Net Margin (%)', 'EBITDA Margin (%)', 
                        'ROE (%)', 'ROA (%)', 'FCF Yield (%)']
    
    # Metrics where LOWER is better (valuation multiples)
    lower_is_better = ['P/E', 'P/B', 'P/S', 'EV/EBITDA', 'PEG']
    
    rankings = {}
    
    print(f"{'Metric':<25} {'Value':<12} {'Percentile':<12} {'Interpretation'}")
    print("-" * 80)
    
    for metric in higher_is_better + lower_is_better:
        if metric not in peer_df.columns:
            continue
            
        target_value = target_data[metric]
        
        if pd.isna(target_value):
            continue
        
        # Calculate percentile
        valid_values = peer_df[metric].dropna()
        
        if len(valid_values) < 2:
            continue
        
        if metric in higher_is_better:
            percentile = stats.percentileofscore(valid_values, target_value, kind='rank')
            interpretation = "Better" if percentile >= 75 else "Worse" if percentile <= 25 else "Average"
        else:
            percentile = 100 - stats.percentileofscore(valid_values, target_value, kind='rank')
            interpretation = "Attractive" if percentile >= 75 else "Expensive" if percentile <= 25 else "Fair"
        
        rankings[metric] = {
            'value': target_value,
            'percentile': percentile,
            'interpretation': interpretation
        }
        
        print(f"{metric:<25} {target_value:>10.2f}  {percentile:>10.1f}%  {interpretation}")
    
    print()
    
    return rankings

# ============================================================
# PART 4: IMPROVED REGRESSION-BASED VALUATION
# ============================================================

def regression_based_valuation(peer_df, target_symbol):
    """
    Use regression to predict fair P/E and EV/EBITDA based on fundamentals
    IMPROVED: Falls back to simpler models and peer averages if insufficient data
    """
    
    print(f"\n{'='*80}")
    print("REGRESSION-BASED FAIR VALUE ANALYSIS")
    print(f"{'='*80}\n")
    
    results = {}
    target_data = peer_df[peer_df['Symbol'] == target_symbol].iloc[0]
    current_price = target_data['Price']
    
    # ============ P/E Model ============
    print("1. P/E Multiple Valuation")
    print("-" * 80)
    
    # Try full regression first
    pe_data = peer_df[peer_df['Symbol'] != target_symbol].copy()
    pe_data = pe_data.dropna(subset=['P/E', 'ROE (%)', 'Revenue Growth (%)', 'Beta'])
    
    predicted_price_pe = None
    
    if len(pe_data) >= 3:  # REDUCED from 5 to 3
        try:
            X = pe_data[['ROE (%)', 'Revenue Growth (%)', 'Beta']].values
            y = pe_data['P/E'].values
            
            model_pe = LinearRegression()
            model_pe.fit(X, y)
            
            if pd.notna(target_data['ROE (%)']) and pd.notna(target_data['Revenue Growth (%)']) and pd.notna(target_data['Beta']):
                X_target = np.array([[
                    target_data['ROE (%)'],
                    target_data['Revenue Growth (%)'],
                    target_data['Beta']
                ]])
                
                predicted_pe = model_pe.predict(X_target)[0]
                actual_pe = target_data['P/E']
                
                # Calculate implied price
                ttm_eps = target_data['Net Income (TTM)'] / (target_data['Market Cap'] / target_data['Price'])
                predicted_price_pe = predicted_pe * ttm_eps
                
                print(f"✓ Regression Model (3 variables)")
                print(f"  P/E = {model_pe.intercept_:.2f} + {model_pe.coef_[0]:.3f}*ROE + {model_pe.coef_[1]:.3f}*Growth + {model_pe.coef_[2]:.3f}*Beta")
                print(f"  Predicted P/E: {predicted_pe:.2f}")
                print(f"  Actual P/E: {actual_pe:.2f}")
                print(f"  Implied Price: ${predicted_price_pe:.2f}")
                print(f"  Current Price: ${current_price:.2f}")
                print(f"  Upside: {((predicted_price_pe - current_price) / current_price * 100):+.1f}%\n")
                
                results['pe_model'] = {
                    'predicted_pe': predicted_pe,
                    'actual_pe': actual_pe,
                    'implied_price': predicted_price_pe,
                    'current_price': current_price
                }
        except:
            pass
    
    # FALLBACK 1: Use peer average P/E if regression failed
    if predicted_price_pe is None:
        peer_pe_avg = peer_df[peer_df['Symbol'] != target_symbol]['P/E'].mean()
        if pd.notna(peer_pe_avg) and pd.notna(target_data['Net Income (TTM)']):
            ttm_eps = target_data['Net Income (TTM)'] / (target_data['Market Cap'] / target_data['Price'])
            predicted_price_pe = peer_pe_avg * ttm_eps
            
            print(f"✓ Peer Average P/E Method (Regression insufficient data)")
            print(f"  Peer Average P/E: {peer_pe_avg:.2f}")
            print(f"  Implied Price: ${predicted_price_pe:.2f}")
            print(f"  Current Price: ${current_price:.2f}")
            print(f"  Upside: {((predicted_price_pe - current_price) / current_price * 100):+.1f}%\n")
            
            results['pe_model'] = {
                'predicted_pe': peer_pe_avg,
                'actual_pe': target_data['P/E'],
                'implied_price': predicted_price_pe,
                'current_price': current_price,
                'method': 'peer_average'
            }
    
    # ============ EV/EBITDA Model ============
    print("2. EV/EBITDA Multiple Valuation")
    print("-" * 80)
    
    ev_data = peer_df[peer_df['Symbol'] != target_symbol].copy()
    ev_data = ev_data.dropna(subset=['EV/EBITDA', 'EBITDA Margin (%)', 'Revenue Growth (%)', 'ROE (%)'])
    
    predicted_price_ev = None
    
    if len(ev_data) >= 3:  # REDUCED from 5 to 3
        try:
            X = ev_data[['EBITDA Margin (%)', 'Revenue Growth (%)', 'ROE (%)']].values
            y = ev_data['EV/EBITDA'].values
            
            model_ev = LinearRegression()
            model_ev.fit(X, y)
            
            if pd.notna(target_data['EBITDA Margin (%)']) and pd.notna(target_data['Revenue Growth (%)']) and pd.notna(target_data['ROE (%)']):
                X_target = np.array([[
                    target_data['EBITDA Margin (%)'],
                    target_data['Revenue Growth (%)'],
                    target_data['ROE (%)']
                ]])
                
                predicted_ev_ebitda = model_ev.predict(X_target)[0]
                actual_ev_ebitda = target_data['EV/EBITDA']
                
                # Calculate implied price
                implied_ev = predicted_ev_ebitda * target_data['EBITDA (TTM)']
                
                ticker = yf.Ticker(target_symbol)
                bs_df = ticker.quarterly_balance_sheet.T.sort_index()
                total_debt = bs_df.iloc[-1].get('Total Debt', 0) or 0
                cash = bs_df.iloc[-1].get('Cash And Cash Equivalents', 0) or 0
                
                implied_equity_value = implied_ev - total_debt + cash
                shares = target_data['Market Cap'] / target_data['Price']
                predicted_price_ev = implied_equity_value / shares
                
                print(f"✓ Regression Model (3 variables)")
                print(f"  EV/EBITDA = {model_ev.intercept_:.2f} + {model_ev.coef_[0]:.3f}*EBITDA_Margin + {model_ev.coef_[1]:.3f}*Growth + {model_ev.coef_[2]:.3f}*ROE")
                print(f"  Predicted EV/EBITDA: {predicted_ev_ebitda:.2f}")
                print(f"  Actual EV/EBITDA: {actual_ev_ebitda:.2f}")
                print(f"  Implied Price: ${predicted_price_ev:.2f}")
                print(f"  Current Price: ${current_price:.2f}")
                print(f"  Upside: {((predicted_price_ev - current_price) / current_price * 100):+.1f}%\n")
                
                results['ev_model'] = {
                    'predicted_ev_ebitda': predicted_ev_ebitda,
                    'actual_ev_ebitda': actual_ev_ebitda,
                    'implied_price': predicted_price_ev,
                    'current_price': current_price
                }
        except:
            pass
    
    # FALLBACK 2: Use peer average EV/EBITDA
    if predicted_price_ev is None:
        peer_ev_avg = peer_df[peer_df['Symbol'] != target_symbol]['EV/EBITDA'].mean()
        if pd.notna(peer_ev_avg) and pd.notna(target_data['EBITDA (TTM)']):
            implied_ev = peer_ev_avg * target_data['EBITDA (TTM)']
            
            ticker = yf.Ticker(target_symbol)
            bs_df = ticker.quarterly_balance_sheet.T.sort_index()
            total_debt = bs_df.iloc[-1].get('Total Debt', 0) or 0
            cash = bs_df.iloc[-1].get('Cash And Cash Equivalents', 0) or 0
            
            implied_equity_value = implied_ev - total_debt + cash
            shares = target_data['Market Cap'] / target_data['Price']
            predicted_price_ev = implied_equity_value / shares
            
            print(f"✓ Peer Average EV/EBITDA Method (Regression insufficient data)")
            print(f"  Peer Average EV/EBITDA: {peer_ev_avg:.2f}")
            print(f"  Implied Price: ${predicted_price_ev:.2f}")
            print(f"  Current Price: ${current_price:.2f}")
            print(f"  Upside: {((predicted_price_ev - current_price) / current_price * 100):+.1f}%\n")
            
            results['ev_model'] = {
                'predicted_ev_ebitda': peer_ev_avg,
                'actual_ev_ebitda': target_data['EV/EBITDA'],
                'implied_price': predicted_price_ev,
                'current_price': current_price,
                'method': 'peer_average'
            }
    
    # ============ Combined Fair Value ============
    prices = []
    if 'pe_model' in results:
        prices.append(results['pe_model']['implied_price'])
    if 'ev_model' in results:
        prices.append(results['ev_model']['implied_price'])
    
    if prices:
        avg_implied_price = np.mean(prices)
        
        print("="*80)
        print("COMBINED FAIR VALUE")
        print("="*80)
        if 'pe_model' in results:
            print(f"P/E Model Fair Price: ${results['pe_model']['implied_price']:.2f}")
        if 'ev_model' in results:
            print(f"EV/EBITDA Model Fair Price: ${results['ev_model']['implied_price']:.2f}")
        print(f"Average Fair Price: ${avg_implied_price:.2f}")
        print(f"Current Price: ${current_price:.2f}")
        print(f"Implied Upside/Downside: {((avg_implied_price - current_price) / current_price * 100):+.1f}%\n")
        
        results['combined_fair_price'] = avg_implied_price
    else:
        print("⚠️ Could not calculate valuation models - insufficient peer data\n")
    
    return results

# ============================================================
# PART 5: COMPREHENSIVE PEER ANALYSIS
# ============================================================

def comprehensive_peer_analysis(symbol, peer_list=None, max_peers=15):
    """
    Complete peer analysis framework
    """
    
    print(f"\n{'='*80}")
    print(f"COMPREHENSIVE PEER ANALYSIS FOR {symbol}")
    print(f"{'='*80}\n")
    
    # Step 1: Select peers if not provided
    if peer_list is None:
        peer_list = get_peer_companies(symbol, max_peers=max_peers)
    
    if not peer_list:
        print("Error: No valid peer companies found")
        return None
    
    # Step 2: Collect data
    peer_df = collect_peer_data(symbol, peer_list)
    
    if peer_df.empty or len(peer_df) < 2:
        print("Error: Insufficient peer data collected")
        return None
    
    # Step 3: Percentile rankings
    rankings = calculate_percentile_rankings(peer_df, symbol)
    
    # Step 4: Regression-based valuation
    regression_results = regression_based_valuation(peer_df, symbol)
    
    # Step 5: Summary comparison table
    print(f"\n{'='*80}")
    print("PEER COMPARISON TABLE")
    print(f"{'='*80}\n")
    
    display_cols = ['Symbol', 'Market Cap', 'P/E', 'P/B', 'EV/EBITDA', 'PEG',
                    'Revenue Growth (%)', 'Net Margin (%)', 'ROE (%)', 'FCF Yield (%)']
    
    display_df = peer_df[display_cols].copy()
    display_df['Market Cap'] = display_df['Market Cap'].apply(lambda x: f"${x/1e9:.1f}B" if pd.notna(x) else "N/A")
    display_df['Symbol'] = display_df['Symbol'].apply(lambda x: f">>> {x} <<<" if x == symbol else x)
    
    print(display_df.to_string(index=False))
    print()
    
    # Export to CSV
    peer_df.to_csv(f'{symbol}_peer_analysis.csv', index=False)
    print(f"✓ Peer analysis exported to {symbol}_peer_analysis.csv\n")
    
    return {
        'symbol': symbol,
        'peer_data': peer_df,
        'rankings': rankings,
        'regression_results': regression_results,
        'peer_list': peer_list
    }

# ============================================================
# USAGE EXAMPLE
# ============================================================

if __name__ == "__main__":
    symbol = "GOOG"
    
    # Run analysis
    results = comprehensive_peer_analysis(symbol, max_peers=15)
    
    if results:
        print("\n" + "="*80)
        print("PEER ANALYSIS COMPLETE")
        print("="*80)
        print(f"\nKey Findings:")
        print(f"- Analyzed {len(results['peer_list'])} peer companies")
        
        # ============================================================
        # FINAL PREDICTED STOCK PRICE
        # ============================================================
        print("\n" + "=" * 40)
        print("FINAL PREDICTED STOCK PRICE")
        print("=" * 40 + "\n")
        
        current_price = results['regression_results'].get('pe_model', {}).get('current_price')
        
        if 'combined_fair_price' in results['regression_results']:
            predicted_price = results['regression_results']['combined_fair_price']
            upside = ((predicted_price - current_price) / current_price * 100) if current_price else 0
            
            print(f"  Stock Symbol: {symbol}")
            print(f"  Current Price: ${current_price:.2f}")
            print(f"  PREDICTED PRICE: ${predicted_price:.2f}")
            print(f"  Expected Return: {upside:+.1f}%")
            
            if upside > 15:
                print(f"  💡 Recommendation: STRONG BUY")
            elif upside > 5:
                print(f"  💡 Recommendation: BUY")
            elif upside > -5:
                print(f"  💡 Recommendation: HOLD")
            elif upside > -15:
                print(f"  💡 Recommendation: REDUCE")
            else:
                print(f"  💡 Recommendation: SELL")
                
        elif 'pe_model' in results['regression_results']:
            predicted_price = results['regression_results']['pe_model']['implied_price']
            upside = ((predicted_price - current_price) / current_price * 100) if current_price else 0
            
            print(f"  Stock Symbol: {symbol}")
            print(f"  Current Price: ${current_price:.2f}")
            print(f"  PREDICTED PRICE: ${predicted_price:.2f}")
            print(f"  Expected Return: {upside:+.1f}%")
            print(f"  (Based on P/E model only)")
            
        elif 'ev_model' in results['regression_results']:
            predicted_price = results['regression_results']['ev_model']['implied_price']
            upside = ((predicted_price - current_price) / current_price * 100) if current_price else 0
            
            print(f"  Stock Symbol: {symbol}")
            print(f"  Current Price: ${current_price:.2f}")
            print(f"  PREDICTED PRICE: ${predicted_price:.2f}")
            print(f"  Expected Return: {upside:+.1f}%")
            print(f"  (Based on EV/EBITDA model only)")
        else:
            print(f"  ⚠️ Unable to calculate predicted price")
            print(f"  Try adding more peer companies or checking data availability")
        
        print("\n" + "=" * 40 + "\n")


COMPREHENSIVE PEER ANALYSIS FOR GOOG


SELECTING PEER COMPANIES FOR GOOG

Target Company: GOOG
Sector: Communication Services
Industry: Internet Content & Information
Market Cap: $4,086,677,897,216

Screening 9 candidates in Communication Services sector...

Selected 2 peer companies:
--------------------------------------------------------------------------------
  GOOGL: Market Cap $4,093,897,080,832 (1.00x)
  META: Market Cap $1,812,426,522,624 (0.44x)


COLLECTING PEER FINANCIAL DATA

Fetching data for GOOG...
Fetching data for GOOGL...
Fetching data for META...

✓ Successfully collected data for 3 companies


PERCENTILE RANKINGS FOR GOOG

Metric                    Value        Percentile   Interpretation
--------------------------------------------------------------------------------
Revenue Growth (%)            336.71        83.3%  Better
Net Margin (%)                 32.23        83.3%  Better
EBITDA Margin (%)              44.55        50.0%  Average
ROE (%)                 

In [14]:
!pip install anthropic
from datetime import datetime, timedelta
import json
import anthropic
from typing import Dict, List, Tuple, Optional

# ============================================================
# CATALYST ANALYSIS FUNCTIONS
# ============================================================

def get_recent_news(symbol: str, months_back: int = 3) -> List[Dict]:
    """Fetch recent news for a stock symbol."""
    try:
        ticker = yf.Ticker(symbol)
        news = ticker.news
        
        if not news:
            print(f"⚠️  No news found for {symbol}")
            return []
        
        cutoff_date = datetime.now() - timedelta(days=months_back * 30)
        filtered_news = []
        
        for article in news[:50]:
            try:
                pub_date = datetime.fromtimestamp(article.get('providerPublishTime', 0))
                if pub_date >= cutoff_date:
                    filtered_news.append({
                        'title': article.get('title', ''),
                        'publisher': article.get('publisher', ''),
                        'link': article.get('link', ''),
                        'date': pub_date.strftime('%Y-%m-%d'),
                        'timestamp': pub_date
                    })
            except Exception:
                continue
        
        return sorted(filtered_news, key=lambda x: x['timestamp'], reverse=True)
    
    except Exception as e:
        print(f"❌ Error fetching news: {str(e)}")
        return []


def identify_catalysts_with_llm(
    symbol: str,
    news_items: List[Dict],
    financial_summary: str,
    api_key: str
) -> Dict:
    """Use Claude API to analyze news and identify key catalysts."""
    client = anthropic.Anthropic(api_key=api_key)
    
    news_text = "\n\n".join([
        f"[{item['date']}] {item['title']} ({item['publisher']})"
        for item in news_items[:20]
    ])
    
    prompt = f"""You are a financial analyst identifying investment catalysts for {symbol}.

FINANCIAL SUMMARY:
{financial_summary}

RECENT NEWS:
{news_text}

Analyze the news and financials to identify catalysts. Return your analysis as a JSON object with this structure:
{{
  "catalysts": [
    {{
      "type": "POSITIVE" or "NEGATIVE" or "NEUTRAL",
      "category": "earnings"|"product"|"regulatory"|"management"|"market"|"partnership"|"other",
      "description": "Brief description",
      "impact_score": number from -10 to +10,
      "timeframe": "short-term"|"medium-term"|"long-term",
      "confidence": "high"|"medium"|"low"
    }}
  ],
  "overall_sentiment": "bullish"|"neutral"|"bearish",
  "key_risks": ["risk1", "risk2"],
  "key_opportunities": ["opp1", "opp2"]
}}

Focus on material catalysts that could significantly impact stock price. Be specific and evidence-based."""
    
    try:
        message = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=2000,
            messages=[{"role": "user", "content": prompt}]
        )
        
        response_text = message.content[0].text
        json_start = response_text.find('{')
        json_end = response_text.rfind('}') + 1
        
        if json_start >= 0 and json_end > json_start:
            json_str = response_text[json_start:json_end]
            return json.loads(json_str)
        else:
            print("⚠️  Could not parse LLM response")
            return {"catalysts": [], "overall_sentiment": "neutral"}
    
    except Exception as e:
        print(f"❌ Error calling Claude API: {str(e)}")
        return {"catalysts": [], "overall_sentiment": "neutral"}


def calculate_catalyst_adjusted_price(
    base_price: float,
    catalysts: List[Dict],
    current_price: float
) -> Tuple[float, Dict]:
    """Calculate weighted price target incorporating catalysts."""
    if not catalysts:
        return base_price, {"base_price": base_price, "catalyst_adjustment_pct": 0}
    
    confidence_weights = {"high": 1.0, "medium": 0.6, "low": 0.3}
    timeframe_weights = {"short-term": 0.8, "medium-term": 1.0, "long-term": 0.5}
    
    total_impact = 0
    for catalyst in catalysts:
        impact = catalyst.get('impact_score', 0)
        confidence = confidence_weights.get(catalyst.get('confidence', 'medium'), 0.6)
        timeframe = timeframe_weights.get(catalyst.get('timeframe', 'medium-term'), 1.0)
        total_impact += impact * confidence * timeframe
    
    # Convert impact to price adjustment (capped at ±20%)
    adjustment_pct = np.clip(total_impact, -20, 20) / 100
    
    # Blend base price and current price (70% base, 30% current)
    blended_base = base_price * 0.7 + current_price * 0.3
    adjusted_price = blended_base * (1 + adjustment_pct)
    
    return adjusted_price, {
        "base_price": base_price,
        "current_price": current_price,
        "blended_base": blended_base,
        "catalyst_adjustment_pct": adjustment_pct * 100,
        "adjusted_price": adjusted_price,
        "num_catalysts": len(catalysts),
        "total_impact_score": total_impact
    }


def generate_investment_recommendation(
    current_price: float,
    target_price: float,
    catalysts: List[Dict],
    sentiment: str
) -> Dict:
    """Generate investment recommendation based on all analysis."""
    upside = ((target_price - current_price) / current_price) * 100
    
    positive_catalysts = sum(1 for c in catalysts if c.get('type') == 'POSITIVE')
    negative_catalysts = sum(1 for c in catalysts if c.get('type') == 'NEGATIVE')
    
    # Base recommendation from upside
    if upside > 20:
        base_rec = "STRONG BUY"
        confidence = "High"
    elif upside > 10:
        base_rec = "BUY"
        confidence = "Medium-High"
    elif upside > -5:
        base_rec = "HOLD"
        confidence = "Medium"
    elif upside > -15:
        base_rec = "REDUCE"
        confidence = "Medium-High"
    else:
        base_rec = "SELL"
        confidence = "High"
    
    # Adjust based on catalysts and sentiment
    if sentiment == "bearish" and negative_catalysts > positive_catalysts:
        if base_rec == "STRONG BUY":
            base_rec = "BUY"
        elif base_rec == "BUY":
            base_rec = "HOLD"
    elif sentiment == "bullish" and positive_catalysts > negative_catalysts:
        if base_rec == "HOLD":
            base_rec = "BUY"
        elif base_rec == "BUY":
            base_rec = "STRONG BUY"
    
    reasons = [
        f"Price target implies {upside:+.1f}% upside/downside",
        f"{positive_catalysts} positive catalyst(s) identified" if positive_catalysts > 0 else None,
        f"{negative_catalysts} negative catalyst(s) identified" if negative_catalysts > 0 else None,
        f"Overall market sentiment is {sentiment}"
    ]
    reasons = [r for r in reasons if r]
    
    return {
        "recommendation": base_rec,
        "confidence": confidence,
        "target_price": target_price,
        "current_price": current_price,
        "expected_return": upside,
        "reasoning": reasons,
        "positive_catalysts": positive_catalysts,
        "negative_catalysts": negative_catalysts,
        "sentiment": sentiment
    }


def generate_investment_memo(
    symbol: str,
    recommendation_data: Dict,
    catalyst_data: Dict,
    company_info: Dict,
    api_key: str
) -> str:
    """Generate a professional investment memo using Claude API."""
    client = anthropic.Anthropic(api_key=api_key)
    
    context = f"""Generate a professional investment memo for {symbol} ({company_info.get('longName', symbol)}).

COMPANY OVERVIEW:
- Sector: {company_info.get('sector', 'N/A')}
- Industry: {company_info.get('industry', 'N/A')}
- Market Cap: ${company_info.get('marketCap', 0) / 1e9:.1f}B

RECOMMENDATION:
- Action: {recommendation_data['recommendation']}
- Target Price: ${recommendation_data['target_price']:.2f}
- Current Price: ${recommendation_data['current_price']:.2f}
- Expected Return: {recommendation_data['expected_return']:.1f}%
- Confidence: {recommendation_data['confidence']}

CATALYSTS:
{json.dumps(catalyst_data.get('catalysts', []), indent=2)}

SENTIMENT: {catalyst_data.get('overall_sentiment', 'neutral')}

KEY RISKS:
{json.dumps(catalyst_data.get('key_risks', []), indent=2)}

KEY OPPORTUNITIES:
{json.dumps(catalyst_data.get('key_opportunities', []), indent=2)}

Write a professional investment memo with these sections:
1. Executive Summary
2. Investment Recommendation
3. Investment Thesis
4. Catalysts Analysis
5. Valuation
6. Risk Factors
7. Financial Health
8. Conclusion

Use professional financial language. Be specific with numbers. Format in clean Markdown."""
    
    try:
        message = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=4000,
            messages=[{"role": "user", "content": context}]
        )
        
        return message.content[0].text
    
    except Exception as e:
        print(f"❌ Error generating memo: {str(e)}")
        return f"# Error Generating Investment Memo\n\nError: {str(e)}"

   ---------------------------------------- 0.0/397.9 kB ? eta -:--:--
   ------ --------------------------------- 61.4/397.9 kB 1.7 MB/s eta 0:00:01
   ----------------- ---------------------- 174.1/397.9 kB 1.7 MB/s eta 0:00:01
   ------------------------- -------------- 256.0/397.9 kB 2.0 MB/s eta 0:00:01
   ----------------------------------- ---- 348.2/397.9 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 397.9/397.9 kB 1.8 MB/s eta 0:00:00


In [19]:
! pip install --upgrade google_genai

     ---------------------------------------- 0.0/53.1 kB ? eta -:--:--
     ------- -------------------------------- 10.2/53.1 kB ? eta -:--:--
     --------------------- ---------------- 30.7/53.1 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 53.1/53.1 kB 343.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/721.9 kB ? eta -:--:--
   --- ------------------------------------ 61.4/721.9 kB 3.2 MB/s eta 0:00:01
   ------- -------------------------------- 143.4/721.9 kB 2.1 MB/s eta 0:00:01
   ----------------- ---------------------- 317.4/721.9 kB 2.8 MB/s eta 0:00:01
   ----------------------- ---------------- 430.1/721.9 kB 2.7 MB/s eta 0:00:01
   ----------------------------- ---------- 532.5/721.9 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------  716.8/721.9 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 721.9/721.9 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/236.5 kB ? eta -:

In [30]:
! pip install google-generativeai

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/155.1 kB ? eta -:--:--
   ------- ------------------------------- 30.7/155.1 kB 660.6 kB/s eta 0:00:01
   ---------- ---------------------------- 41.0/155.1 kB 393.8 kB/s eta 0:00:01
   -------------------- ------------------ 81.9/155.1 kB 657.6 kB/s eta 0:00:01
   --------------------------- ---------- 112.6/155.1 kB 547.6 kB/s eta 0:00:01
   -------------------------------------- 155.1/155.1 kB 579.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
    --------------------------------------- 0.0/1.3 MB 1.4 MB/s eta 0:00:01
   - ----

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.5 which is incompatible.
streamlit 1.32.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.


In [38]:
"""
Enhanced Investment Analysis Pipeline - Gemini Version
Uses Google's Gemini API instead of Anthropic Claude

INSTALLATION REQUIRED:
Run this command first:
    pip install google-generativeai

or if using conda:
    conda install -c conda-forge google-generativeai
"""

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import google.generativeai as genai
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')


def get_recent_news(symbol: str, months_back: int = 3) -> List[Dict]:
    """Fetch recent news for a stock symbol."""
    try:
        ticker = yf.Ticker(symbol)
        news = ticker.news
        
        if not news:
            print(f"Warning: No news found for {symbol}")
            return []
        
        cutoff_date = datetime.now() - timedelta(days=months_back * 30)
        filtered_news = []
        
        for article in news[:50]:
            try:
                pub_date = datetime.fromtimestamp(article.get('providerPublishTime', 0))
                if pub_date >= cutoff_date:
                    filtered_news.append({
                        'title': article.get('title', ''),
                        'publisher': article.get('publisher', ''),
                        'link': article.get('link', ''),
                        'date': pub_date.strftime('%Y-%m-%d'),
                        'timestamp': pub_date
                    })
            except Exception:
                continue
        
        return sorted(filtered_news, key=lambda x: x['timestamp'], reverse=True)
    
    except Exception as e:
        print(f"Error fetching news: {str(e)}")
        return []


def identify_catalysts_with_gemini(
    symbol: str,
    news_items: List[Dict],
    financial_summary: str,
    api_key: str
) -> Dict:
    """Use Gemini API to analyze news and identify key catalysts."""
    
    if not news_items:
        print("Warning: No news items to analyze. Skipping catalyst identification.")
        return {
            "catalysts": [],
            "overall_sentiment": "neutral",
            "key_risks": ["Limited news coverage available"],
            "key_opportunities": []
        }
    
    try:
        # Configure Gemini
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-pro')
    except Exception as e:
        print(f"Error initializing Gemini client: {str(e)}")
        print("Please check your API key at https://makersuite.google.com/app/apikey")
        return {"catalysts": [], "overall_sentiment": "neutral", "key_risks": [], "key_opportunities": []}
    
    news_text = "\n\n".join([
        f"[{item['date']}] {item['title']} ({item['publisher']})"
        for item in news_items[:20]
    ])
    
    prompt = f"""You are a financial analyst identifying investment catalysts for {symbol}.

FINANCIAL SUMMARY:
{financial_summary}

RECENT NEWS:
{news_text}

Analyze the news and financials to identify catalysts. Return your analysis as a JSON object with this structure:
{{
  "catalysts": [
    {{
      "type": "POSITIVE" or "NEGATIVE" or "NEUTRAL",
      "category": "earnings"|"product"|"regulatory"|"management"|"market"|"partnership"|"other",
      "description": "Brief description",
      "impact_score": number from -10 to +10,
      "timeframe": "short-term"|"medium-term"|"long-term",
      "confidence": "high"|"medium"|"low"
    }}
  ],
  "overall_sentiment": "bullish"|"neutral"|"bearish",
  "key_risks": ["risk1", "risk2"],
  "key_opportunities": ["opp1", "opp2"]
}}

Focus on material catalysts that could significantly impact stock price. Be specific and evidence-based.
Return ONLY valid JSON, no additional text or markdown formatting."""
    
    try:
        response = model.generate_content(prompt)
        response_text = response.text
        
        # Clean up response text
        response_text = response_text.strip()
        
        # Remove markdown code blocks if present
        if response_text.startswith('```json'):
            response_text = response_text[7:]
        if response_text.startswith('```'):
            response_text = response_text[3:]
        if response_text.endswith('```'):
            response_text = response_text[:-3]
        
        response_text = response_text.strip()
        
        # Find JSON object
        json_start = response_text.find('{')
        json_end = response_text.rfind('}') + 1
        
        if json_start >= 0 and json_end > json_start:
            json_str = response_text[json_start:json_end]
            result = json.loads(json_str)
            return result
        else:
            print("Warning: Could not parse Gemini response as JSON")
            return {"catalysts": [], "overall_sentiment": "neutral", "key_risks": [], "key_opportunities": []}
    
    except Exception as e:
        print(f"Error calling Gemini API: {str(e)}")
        print("\nPlease verify:")
        print("1. Your API key is correct")
        print("2. Gemini API is enabled for your project")
        print("3. You have available quota")
        return {"catalysts": [], "overall_sentiment": "neutral", "key_risks": [], "key_opportunities": []}


def calculate_catalyst_adjusted_price(
    base_price: float,
    catalysts: List[Dict],
    current_price: float
) -> Tuple[float, Dict]:
    """Calculate weighted price target incorporating catalysts."""
    if not catalysts:
        return base_price, {
            "base_price": base_price,
            "current_price": current_price,
            "blended_base": base_price,
            "catalyst_adjustment_pct": 0,
            "adjusted_price": base_price,
            "num_catalysts": 0,
            "total_impact_score": 0
        }
    
    confidence_weights = {"high": 1.0, "medium": 0.6, "low": 0.3}
    timeframe_weights = {"short-term": 0.8, "medium-term": 1.0, "long-term": 0.5}
    
    total_impact = 0
    for catalyst in catalysts:
        impact = catalyst.get('impact_score', 0)
        confidence = confidence_weights.get(catalyst.get('confidence', 'medium'), 0.6)
        timeframe = timeframe_weights.get(catalyst.get('timeframe', 'medium-term'), 1.0)
        total_impact += impact * confidence * timeframe
    
    adjustment_pct = np.clip(total_impact, -20, 20) / 100
    blended_base = base_price * 0.7 + current_price * 0.3
    adjusted_price = blended_base * (1 + adjustment_pct)
    
    return adjusted_price, {
        "base_price": base_price,
        "current_price": current_price,
        "blended_base": blended_base,
        "catalyst_adjustment_pct": adjustment_pct * 100,
        "adjusted_price": adjusted_price,
        "num_catalysts": len(catalysts),
        "total_impact_score": total_impact
    }


def generate_investment_recommendation(
    current_price: float,
    target_price: float,
    catalysts: List[Dict],
    sentiment: str
) -> Dict:
    """Generate investment recommendation based on all analysis."""
    upside = ((target_price - current_price) / current_price) * 100
    
    positive_catalysts = sum(1 for c in catalysts if c.get('type') == 'POSITIVE')
    negative_catalysts = sum(1 for c in catalysts if c.get('type') == 'NEGATIVE')
    
    if upside > 20:
        base_rec = "STRONG BUY"
        confidence = "High"
    elif upside > 10:
        base_rec = "BUY"
        confidence = "Medium-High"
    elif upside > -5:
        base_rec = "HOLD"
        confidence = "Medium"
    elif upside > -15:
        base_rec = "REDUCE"
        confidence = "Medium-High"
    else:
        base_rec = "SELL"
        confidence = "High"
    
    if sentiment == "bearish" and negative_catalysts > positive_catalysts:
        if base_rec == "STRONG BUY":
            base_rec = "BUY"
        elif base_rec == "BUY":
            base_rec = "HOLD"
    elif sentiment == "bullish" and positive_catalysts > negative_catalysts:
        if base_rec == "HOLD":
            base_rec = "BUY"
        elif base_rec == "BUY":
            base_rec = "STRONG BUY"
    
    reasons = [
        f"Price target implies {upside:+.1f}% upside/downside",
        f"{positive_catalysts} positive catalyst(s) identified" if positive_catalysts > 0 else None,
        f"{negative_catalysts} negative catalyst(s) identified" if negative_catalysts > 0 else None,
        f"Overall market sentiment is {sentiment}"
    ]
    reasons = [r for r in reasons if r]
    
    return {
        "recommendation": base_rec,
        "confidence": confidence,
        "target_price": target_price,
        "current_price": current_price,
        "expected_return": upside,
        "reasoning": reasons,
        "positive_catalysts": positive_catalysts,
        "negative_catalysts": negative_catalysts,
        "sentiment": sentiment
    }


def generate_investment_memo_with_gemini(
    symbol: str,
    recommendation_data: Dict,
    catalyst_data: Dict,
    company_info: Dict,
    api_key: str
) -> str:
    """Generate a professional investment memo using Gemini API."""
    
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-pro')
    except Exception as e:
        return f"# Error: Unable to generate memo\n\nInvalid API key. Please check your Gemini API key.\n\nError: {str(e)}"
    
    context = f"""Generate a professional investment memo for {symbol} ({company_info.get('longName', symbol)}).

COMPANY OVERVIEW:
- Sector: {company_info.get('sector', 'N/A')}
- Industry: {company_info.get('industry', 'N/A')}
- Market Cap: ${company_info.get('marketCap', 0) / 1e9:.1f}B

RECOMMENDATION:
- Action: {recommendation_data['recommendation']}
- Target Price: ${recommendation_data['target_price']:.2f}
- Current Price: ${recommendation_data['current_price']:.2f}
- Expected Return: {recommendation_data['expected_return']:.1f}%
- Confidence: {recommendation_data['confidence']}

CATALYSTS:
{json.dumps(catalyst_data.get('catalysts', []), indent=2)}

SENTIMENT: {catalyst_data.get('overall_sentiment', 'neutral')}

KEY RISKS:
{json.dumps(catalyst_data.get('key_risks', []), indent=2)}

KEY OPPORTUNITIES:
{json.dumps(catalyst_data.get('key_opportunities', []), indent=2)}

KEY FINANCIAL METRICS:
- P/E Ratio: {company_info.get('trailingPE', 'N/A')}
- Forward P/E: {company_info.get('forwardPE', 'N/A')}
- Profit Margin: {company_info.get('profitMargins', 'N/A')}
- ROE: {company_info.get('returnOnEquity', 'N/A')}

Write a professional investment memo with these sections:
1. Executive Summary
2. Investment Recommendation
3. Investment Thesis (3-5 key points)
4. Catalysts Analysis
5. Valuation
6. Risk Factors
7. Financial Health
8. Conclusion

Use professional financial language. Be specific with numbers. Format in clean Markdown.
Do not use any emoji or special characters, use plain text only."""
    
    try:
        response = model.generate_content(context)
        return response.text
    
    except Exception as e:
        return f"# Error Generating Investment Memo\n\nError: {str(e)}"


def complete_investment_analysis(
    symbol: str,
    gemini_api_key: str,
    news_months_back: int = 3,
    base_fair_value: Optional[float] = None
) -> Dict:
    """
    Run complete investment analysis pipeline using Gemini.
    
    Args:
        symbol: Stock ticker
        gemini_api_key: Your Google Gemini API key
        news_months_back: Months of news to analyze
        base_fair_value: Optional base valuation from your peer analysis
    """
    
    print("\n" + "="*80)
    print(f"COMPLETE INVESTMENT ANALYSIS FOR {symbol}")
    print("="*80 + "\n")
    
    # Initialize
    ticker = yf.Ticker(symbol)
    company_info = ticker.info
    current_price = company_info.get('currentPrice', company_info.get('regularMarketPrice', 0))
    
    if current_price == 0:
        print("Error: Could not fetch current price")
        return {}
    
    print(f"Company: {company_info.get('longName', symbol)}")
    print(f"Current Price: ${current_price:.2f}")
    print(f"Market Cap: ${company_info.get('marketCap', 0) / 1e9:.1f}B\n")
    
    # Step 1: Get News
    print("Step 1: Fetching recent news...")
    news_items = get_recent_news(symbol, news_months_back)
    print(f"   Found {len(news_items)} recent articles\n")
    
    # Step 2: Identify Catalysts
    print("Step 2: Identifying catalysts with Gemini AI...")
    financial_summary = f"""
Current Price: ${current_price:.2f}
Market Cap: ${company_info.get('marketCap', 0) / 1e9:.1f}B
P/E Ratio: {company_info.get('trailingPE', 'N/A')}
Forward P/E: {company_info.get('forwardPE', 'N/A')}
Profit Margin: {company_info.get('profitMargins', 'N/A')}
ROE: {company_info.get('returnOnEquity', 'N/A')}
"""
    
    catalyst_data = identify_catalysts_with_gemini(
        symbol, news_items, financial_summary, gemini_api_key
    )
    
    num_catalysts = len(catalyst_data.get('catalysts', []))
    print(f"   Identified {num_catalysts} catalysts")
    print(f"   Overall sentiment: {catalyst_data.get('overall_sentiment', 'neutral').upper()}\n")
    
    # Step 3: Calculate Price Target
    print("Step 3: Calculating catalyst-adjusted price target...")
    
    # Use provided base value or calculate simple valuation
    if base_fair_value is not None:
        base_price = base_fair_value
    else:
        pe_ratio = company_info.get('forwardPE', company_info.get('trailingPE', 20))
        eps = company_info.get('trailingEps', current_price / pe_ratio)
        base_price = eps * pe_ratio * 1.1
    
    adjusted_price, price_breakdown = calculate_catalyst_adjusted_price(
        base_price, catalyst_data.get('catalysts', []), current_price
    )
    
    print(f"   Base Valuation: ${price_breakdown['base_price']:.2f}")
    print(f"   Catalyst Adjustment: {price_breakdown['catalyst_adjustment_pct']:+.1f}%")
    print(f"   Final Target Price: ${adjusted_price:.2f}\n")
    
   

In [39]:
def calculate_final_price(weighted_fair_value, predicted_price, current_price):
    """
    Calculate final predicted price with adaptive weights based on model agreement
    """
    
    # Calculate relative difference between two model predictions
    price_diff = abs(weighted_fair_value - predicted_price)
    relative_diff = price_diff / current_price if current_price > 0 else 0
    
    # Determine weights based on agreement level
    if relative_diff < 0.05:  # Less than 5% difference - high agreement
        w1, w2 = 0.5, 0.5  # Equal weights
        weighting_method = "Equal weights (high agreement between models)"
    
    elif relative_diff < 0.15:  # 5-15% difference - moderate agreement
        w1, w2 = 0.6, 0.4  # Slight preference for DCF
        weighting_method = "Slight DCF preference (moderate agreement)"
    
    else:  # More than 15% difference - significant disagreement
        # Check which model is closer to current market price
        dcf_distance = abs(weighted_fair_value - current_price)
        peer_distance = abs(predicted_price - current_price)
        
        if dcf_distance < peer_distance:
            w1, w2 = 0.7, 0.3  # DCF closer to market price
            weighting_method = "Strong DCF preference (DCF closer to current market price)"
        else:
            w1, w2 = 0.4, 0.6  # Peer analysis closer to market price
            weighting_method = "Strong peer preference (peer analysis closer to current market price)"
    
    # Calculate final price
    final_price = w1 * weighted_fair_value + w2 * predicted_price
    
    return {
        'final_price': final_price,
        'dcf_weight': w1,
        'peer_weight': w2,
        'weighting_method': weighting_method,
        'relative_difference_pct': relative_diff * 100,
        'dcf_value': weighted_fair_value,
        'peer_value': predicted_price
    }


# Alternative: Fixed weighting approach
def calculate_final_price_fixed(weighted_fair_value, predicted_price, weighting_scheme='balanced'):
    """
    Calculate final price using fixed weighting schemes
    """
    
    weighting_schemes = {
        'dcf_dominant': (0.7, 0.3),     # Strong preference for DCF
        'balanced': (0.6, 0.4),         # Balanced with slight DCF preference
        'equal': (0.5, 0.5),            # Equal weights
        'market_aware': (0.55, 0.45),   # Slight market adjustment
    }
    
    w1, w2 = weighting_schemes.get(weighting_scheme, (0.6, 0.4))
    final_price = w1 * weighted_fair_value + w2 * predicted_price
    
    return {
        'final_price': final_price,
        'dcf_weight': w1,
        'peer_weight': w2,
        'weighting_scheme': weighting_scheme
    }


# Complete integration function
def integrate_all_valuations(symbol, weighted_fair_value, predicted_price):
    """
    Main function to integrate all valuation models
    """
    
    print(f"\n{'='*80}")
    print(f"FINAL INTEGRATED VALUATION FOR {symbol}")
    print(f"{'='*80}\n")
    
    # Get current price
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info
        current_price = info.get('currentPrice', info.get('regularMarketPrice'))
    except:
        current_price = (weighted_fair_value + predicted_price) / 2
        print(f"⚠️ Could not fetch current price, using average: ${current_price:.2f}")
    
    # Display input values
    print("MODEL INPUTS:")
    print("-" * 60)
    print(f"DCF (Management-adjusted weighted average): ${weighted_fair_value:.2f}")
    print(f"Peer Analysis (Comparative valuation):      ${predicted_price:.2f}")
    print(f"Current Market Price:                       ${current_price:.2f}")
    
    # Calculate using adaptive weighting
    result = calculate_final_price(weighted_fair_value, predicted_price, current_price)
    
    print(f"\nADAPTIVE WEIGHTING RESULT:")
    print("-" * 60)
    print(f"Relative difference between models: {result['relative_difference_pct']:.1f}%")
    print(f"Weighting method: {result['weighting_method']}")
    print(f"DCF weight: {result['dcf_weight']*100:.0f}%")
    print(f"Peer weight: {result['peer_weight']*100:.0f}%")
    print(f"\nFINAL PREDICTED PRICE: ${result['final_price']:.2f}")
    
    # Calculate upside/downside
    upside = ((result['final_price'] - current_price) / current_price) * 100 if current_price else 0
    print(f"Expected return: {upside:+.1f}%")
    
    # Recommendation
    if upside > 20:
        recommendation = "STRONG BUY"
    elif upside > 10:
        recommendation = "BUY"
    elif upside > -5:
        recommendation = "HOLD"
    elif upside > -15:
        recommendation = "REDUCE"
    else:
        recommendation = "SELL"
    
    print(f"Recommendation: {recommendation}")
    
    # Compare different weighting schemes
    print(f"\n{'='*80}")
    print("WEIGHTING SCHEME COMPARISON")
    print(f"{'='*80}\n")
    
    schemes = ['dcf_dominant', 'balanced', 'equal', 'market_aware']
    comparison = []
    
    for scheme in schemes:
        fixed_result = calculate_final_price_fixed(weighted_fair_value, predicted_price, scheme)
        fixed_upside = ((fixed_result['final_price'] - current_price) / current_price) * 100 if current_price else 0
        comparison.append({
            'scheme': scheme,
            'price': fixed_result['final_price'],
            'upside': fixed_upside,
            'dcf_weight': fixed_result['dcf_weight']*100,
            'peer_weight': fixed_result['peer_weight']*100
        })
    
    # Display comparison table
    print(f"{'Scheme':<15} {'DCF Weight':<12} {'Peer Weight':<12} {'Price':<12} {'Upside':<12}")
    print("-" * 63)
    for comp in comparison:
        print(f"{comp['scheme']:<15} {comp['dcf_weight']:<11.0f}% {comp['peer_weight']:<11.0f}% "
              f"${comp['price']:<11.2f} {comp['upside']:+.1f}%")
    
    print(f"\n{'='*80}")
    print("ANALYSIS COMPLETE")
    print(f"{'='*80}")
    
    # Export results
    export_results(symbol, result, comparison, current_price)
    
    return result


def export_results(symbol, adaptive_result, comparison_results, current_price):
    """
    Export results to CSV
    """
    import pandas as pd
    from datetime import datetime
    
    # Create summary DataFrame
    summary_data = {
        'Metric': [
            'Current Price',
            'DCF Fair Value',
            'Peer Analysis Value', 
            'Adaptive Final Price',
            'Expected Return (%)',
            'Weighting Method',
            'DCF Weight (%)',
            'Peer Weight (%)',
            'Analysis Date'
        ],
        'Value': [
            f"${current_price:.2f}",
            f"${adaptive_result['dcf_value']:.2f}",
            f"${adaptive_result['peer_value']:.2f}",
            f"${adaptive_result['final_price']:.2f}",
            f"{((adaptive_result['final_price'] - current_price) / current_price * 100):+.1f}%",
            adaptive_result['weighting_method'],
            f"{adaptive_result['dcf_weight']*100:.1f}%",
            f"{adaptive_result['peer_weight']*100:.1f}%",
            datetime.now().strftime('%Y-%m-%d')
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame(comparison_results)
    
    # Export to CSV
    summary_df.to_csv(f'{symbol}_integrated_valuation.csv', index=False)
    comparison_df.to_csv(f'{symbol}_weighting_comparison.csv', index=False)
    
    print(f"✓ Results exported to {symbol}_integrated_valuation.csv")
    print(f"✓ Weighting comparison exported to {symbol}_weighting_comparison.csv")


# Usage example
if __name__ == "__main__":
    symbol = "GOOG"
    
    # These should come from your previous analyses
    weighted_fair_value = 324.78  # From DCF with management quality adjustment
    predicted_price = 311.64      # From peer analysis
    
    # Run integrated valuation
    result = integrate_all_valuations(symbol, weighted_fair_value, predicted_price)


FINAL INTEGRATED VALUATION FOR GOOG

MODEL INPUTS:
------------------------------------------------------------
DCF (Management-adjusted weighted average): $324.78
Peer Analysis (Comparative valuation):      $311.64
Current Market Price:                       $338.53

ADAPTIVE WEIGHTING RESULT:
------------------------------------------------------------
Relative difference between models: 3.9%
Weighting method: Equal weights (high agreement between models)
DCF weight: 50%
Peer weight: 50%

FINAL PREDICTED PRICE: $318.21
Expected return: -6.0%
Recommendation: REDUCE

WEIGHTING SCHEME COMPARISON

Scheme          DCF Weight   Peer Weight  Price        Upside      
---------------------------------------------------------------
dcf_dominant    70         % 30         % $320.84      -5.2%
balanced        60         % 40         % $319.52      -5.6%
equal           50         % 50         % $318.21      -6.0%
market_aware    55         % 45         % $318.87      -5.8%

ANALYSIS COMPLETE
✓

In [51]:
# ===============================
# Step 0: Set Gemini API Key
# ===============================
GEMINI_API_KEY = "*"

# ===============================
# Step 4: Generate Investment Recommendation
# ===============================
print("Step 4: Generating investment recommendation...")
recommendation = generate_investment_recommendation(
    current_price,
    adjusted_price,  # Keep original logic
    catalyst_data.get('catalysts', []),
    catalyst_data.get('overall_sentiment', 'neutral')
)

print("\n" + "="*80)
print(f"RECOMMENDATION: {recommendation['recommendation']}")
print("="*80)

# ===============================
# Step 4.5: Run Integrated Valuation to get final target price
# ===============================
adaptive_result = integrate_all_valuations(symbol, weighted_fair_value, predicted_price)
final_target_price = adaptive_result['final_price']  # ← Use this as the real target price

print(f"Target Price: ${final_target_price:.2f}")
print(f"Expected Return: {recommendation['expected_return']:+.1f}%")
print(f"Confidence: {recommendation['confidence']}")

# Simple valuation breakdown
print("\nVALUATION COMPONENTS:")
print("-" * 60)
print(f"• DCF Weighted Fair Value: ${weighted_fair_value:.2f}")
print(f"• Peer Analysis Price: ${predicted_price:.2f}")
print(f"• Integrated Final Price: ${final_target_price:.2f}")
print()

print("Key Reasoning:")
for reason in recommendation['reasoning']:
    print(f"   - {reason}")
print()

# ===============================
# Step 5: Display Catalysts
# ===============================
if catalyst_data.get('catalysts'):
    print("="*80)
    print("KEY CATALYSTS")
    print("="*80 + "\n")
    
    for i, catalyst in enumerate(catalyst_data.get('catalysts', []), 1):
        prefix = "[+]" if catalyst['type'] == "POSITIVE" else "[-]" if catalyst['type'] == "NEGATIVE" else "[~]"
        print(f"{i}. {prefix} {catalyst['description']}")
        print(f"   Type: {catalyst['type']} | Category: {catalyst['category']}")
        print(f"   Impact: {catalyst['impact_score']:+.1f}/10 | {catalyst['timeframe']} | {catalyst['confidence']} confidence\n")

# Display risks and opportunities
if catalyst_data.get('key_risks'):
    print("KEY RISKS:")
    for risk in catalyst_data['key_risks']:
        print(f"   - {risk}")
    print()

if catalyst_data.get('key_opportunities'):
    print("KEY OPPORTUNITIES:")
    for opp in catalyst_data['key_opportunities']:
        print(f"   - {opp}")
    print()

# ===============================
# Step 6: Generate Detailed Investment Memo using Gemini
# ===============================
print("Step 5: Generating professional investment memo with Gemini...")

def generate_detailed_investment_memo_with_gemini(symbol, recommendation, catalyst_data, company_info, api_key, 
                                                  weighted_fair_value, predicted_price, final_price):
    """
    Generate a detailed investment memo (250-350 words)
    """
    detailed_context = f"""Generate a professional investment memo for {symbol} ({company_info.get('longName', symbol)}).

COMPANY OVERVIEW:
- Sector: {company_info.get('sector', 'N/A')}
- Industry: {company_info.get('industry', 'N/A')}
- Market Cap: ${company_info.get('marketCap', 0) / 1e9:.1f}B
- P/E Ratio: {company_info.get('trailingPE', 'N/A')}

VALUATION METHODOLOGY:
1. DCF Model (Intrinsic Value): ${weighted_fair_value:.2f}
2. Peer Comparison (Relative Value): ${predicted_price:.2f}
3. Integrated Result: ${final_price:.2f}

RECOMMENDATION:
- Action: {recommendation['recommendation']}
- Target Price: ${final_price:.2f}  # ← use real final predicted price
- Current Price: ${recommendation['current_price']:.2f}
- Expected Return: {recommendation['expected_return']:.1f}%
- Confidence: {recommendation['confidence']}

CATALYSTS ({len(catalyst_data.get('catalysts', []))} identified):
- Overall Sentiment: {catalyst_data.get('overall_sentiment', 'neutral').upper()}
- Positive: {sum(1 for c in catalyst_data.get('catalysts', []) if c.get('type') == 'POSITIVE')}
- Negative: {sum(1 for c in catalyst_data.get('catalysts', []) if c.get('type') == 'NEGATIVE')}

Write a professional investment memo with sections:
1. EXECUTIVE SUMMARY
2. INVESTMENT THESIS
3. VALUATION ANALYSIS
4. CATALYSTS & RISKS
5. FINANCIAL HEALTH
6. RECOMMENDATION & ACTION PLAN

Requirements:
- Length: 250-350 words
- Professional tone
- Include numbers and valuation methodology"""

    try:
        import google.generativeai as genai
        genai.configure(api_key=api_key)

        # Use a valid Gemini model
        model = genai.GenerativeModel("models/gemini-2.5-flash")

        response = model.generate_content(
            detailed_context,
            generation_config={
                "temperature": 0.7,
                "max_output_tokens": 1500,
                "top_p": 0.9
            }
        )
        return response.text

    except Exception as e:
        print(f"❌ Error generating memo: {str(e)}")
        return f"# Investment Memo for {symbol}\n\nError: {str(e)}"

# Call the function
memo = generate_detailed_investment_memo_with_gemini(
    symbol,
    recommendation,
    catalyst_data,
    company_info,
    GEMINI_API_KEY,
    weighted_fair_value,
    predicted_price,
    final_target_price  # ← use real final predicted price here
)

# Save output
from datetime import datetime
import json

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
memo_filename = f"{symbol}_investment_memo_{timestamp}.md"
with open(memo_filename, 'w') as f:
    f.write(memo)
print(f"   Investment memo saved: {memo_filename}")

results = {
    'symbol': symbol,
    'company_name': company_info.get('longName', symbol),
    'analysis_date': datetime.now().isoformat(),
    'current_price': current_price,
    'target_price': final_target_price,  # ← updated
    'recommendation': recommendation,
    'catalysts': catalyst_data,
    'price_breakdown': price_breakdown,
    'news_count': len(news_items),
    'valuation_components': {
        'dcf_value': weighted_fair_value,
        'peer_value': predicted_price,
        'final_integrated_value': final_target_price
    }
}

results_filename = f"{symbol}_analysis_results_{timestamp}.json"
with open(results_filename, 'w') as f:
    json.dump(results, f, indent=2, default=str)
print(f"   Detailed results saved: {results_filename}\n")

print("="*80)
print("ANALYSIS COMPLETE")
print("="*80 + "\n")

# Return results dictionary
results


Step 4: Generating investment recommendation...

RECOMMENDATION: HOLD

FINAL INTEGRATED VALUATION FOR GOOG

MODEL INPUTS:
------------------------------------------------------------
DCF (Management-adjusted weighted average): $324.78
Peer Analysis (Comparative valuation):      $311.64
Current Market Price:                       $338.53

ADAPTIVE WEIGHTING RESULT:
------------------------------------------------------------
Relative difference between models: 3.9%
Weighting method: Equal weights (high agreement between models)
DCF weight: 50%
Peer weight: 50%

FINAL PREDICTED PRICE: $318.21
Expected return: -6.0%
Recommendation: REDUCE

WEIGHTING SCHEME COMPARISON

Scheme          DCF Weight   Peer Weight  Price        Upside      
---------------------------------------------------------------
dcf_dominant    70         % 30         % $320.84      -5.2%
balanced        60         % 40         % $319.52      -5.6%
equal           50         % 50         % $318.21      -6.0%
market_awar

{'symbol': 'GOOG',
 'company_name': 'Alphabet Inc.',
 'analysis_date': '2026-02-01T09:21:56.747961',
 'current_price': 338.53,
 'target_price': 318.21,
 'recommendation': {'recommendation': 'HOLD',
  'confidence': 'Medium',
  'target_price': 334.26312435999995,
  'current_price': 338.53,
  'expected_return': -1.2604128555814915,
  'reasoning': ['Price target implies -1.3% upside/downside',
   'Overall market sentiment is neutral'],
  'positive_catalysts': 0,
  'negative_catalysts': 0,
  'sentiment': 'neutral'},
 'catalysts': {'catalysts': [], 'overall_sentiment': 'neutral'},
 'price_breakdown': {'base_price': 334.26312435999995,
  'catalyst_adjustment_pct': 0},
 'news_count': 0,
 'valuation_components': {'dcf_value': 324.78,
  'peer_value': 311.64,
  'final_integrated_value': 318.21}}

In [50]:
import os
print(os.getcwd())


C:\Users\HUAWEI
